In [1]:
# DeepOPF-Dis: distributed DeepOPF-V
# 2024-3-18
# 2024-3-24: no constraint penalty and no kron reduction
# 2024-3-25: no constraint penalty but add kron reduction
# 2024-4-6: use multi-branch DNN supervised learning
# 2024-4-6: add penalty 
# 2024-4-7: simplify the code
# 2024-4-10: single branch DNN
import torch
import numpy as np
import scipy.io
import torch.nn as nn
import torch.nn.functional as F
import time
import math 
import torch.utils.data as Data
from torch.autograd import Function
from scipy import sparse
from torch.nn.utils import weight_norm as wn
import gc
import random
import matplotlib.pyplot as plt
# from torchviz import make_dot
random.seed(12343)

global BRANFT, branch,finc,tinc,Nbranch 
global Ybus,baseMVA,baseMVA_tensor
global Vscale, Vbias, bus_Pg, bus_Qg, bus_Pd, bus_Qd, bus_Pnet_nonPg, bus_Pnet_nonQg, bus_Va
global flagV,DELTA,Nbus,Ntest,Ng,Nd
global idxPg_tensor, gencost_tensor
global kcost, kpd, kqd, kgenp, kgenq, kbpf, kbang, kv
global MAXMIN_PQg, MAXMIN_Pg_tensor, MAXMIN_Qg_tensor, MAXMIN_PQgall_tensor
global MAXMIN_Pg, MAXMIN_Qg, MAXMIN_PQgall
global kpd_max,kqd_max,kgenp_max,kgenq_max, kv_max
global flag_k 
global idx_bus_Pnet_Vam_sort

In [2]:
## print('*'*10,'L2 sigmoid','*'*10)

## whether there is GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device',device)
print("Let's use", torch.cuda.device_count(), "GPUs!")
    
# 控制打印格式保留小数点n位
torch.set_printoptions(precision=4)
np.set_printoptions(precision=4)

## load system name
version = 1 # number of model
T = 1 # running the Tth time
# Nbus = 118
# sys_R = 4 # case300:[0, 1, 2]; case118[0, 1]; case30[0, 1]

Nbus = 30
sys_R = 0 # case300:[0, 1, 2]; case118[0, 1]; case30[0, 1]


flag_test = 1 # test
flag_save_ls = 0 # save values of loss function
s_epoch = 3000 # save trainged model during training
p_epoch = 100 # print loss function value
Epoch = 4500
Nsample = 50000 # number of samples: including training, test and post-processing samples
Nhis = 3 # number of sampels used for post-processing
Ntrain = 32000
Ntest = 10000
batch_size_training = 50
batch_size_test = 1

## DNN structure: fully connected layers
hidden_units = 1
# khidden = np.array([256,336], dtype=int)
khidden = np.array([32,224], dtype=int)
# khidden = np.array([256,128], dtype=int)

nmL = 'L'
for i in range(khidden.shape[0]):
    nmL = nmL + str(khidden[i])  

Lr = 1e-4
DELTA = 1e-4 # threshold of violation
kdelta = 1e-4 # threshold of violation in training
    

## loss function coefficients
flag_k = 1

kpy = 1 # penalty for constraint violation
if kpy > 0 :
    method = '_krp' # kron reduction & penlaty
else:
    method = '_krngtp' # kron reduction & penlaty

kpy = torch.tensor([kpy])
    
kcost = torch.tensor([0.0002]) # scaling of Pg cost
Ki = torch.tensor([100.0]) # tensor coefficent
# Ki = torch.tensor([1.0]) # tensor coefficent
kpdi = 100.0 # P load
kqdi = 100.0 # Q load
kgenpi = 2000.0 # Pg
kgenqi = 2000.0 # Qg
kvi = 100.0 # voltage violation for recovered ZIBs 
k_dV = 0.1 # post-processing coefficient: step of delta_Va delta_Vm

# upper bound of loss function coefficients
kpd_max = torch.tensor([100.0])
kqd_max = torch.tensor([100.0])
kgenp_max = torch.tensor([2000.0])
kgenq_max = torch.tensor([2000.0])
kv_max = torch.tensor([500.0])

# save loss function value during training  
ls_cost_log = []
ls_Pg_log = []
ls_Qg_log = []
ls_Pd_log = []
ls_Qd_log = []

k_cost_log = []
k_Pg_log = []
k_Qg_log = []
k_Pd_log = []
k_Qd_log = []

# file names
kcoef = 'B'+str(Nbus)+'r'+str(sys_R)+'V'+str(version)+'S'+str(int(Ntrain))+'Nhis'+str(int(Nhis))+'lr'+str(Lr)+'kc'+str(round(float(kcost),2))+'kd'+str(int(kpd_max))+\
'kg'+str(int(kgenp_max))+'kv'+str(int(kv_max))+'ky'+str(int(kpy))+nmL+'bs'+str(batch_size_training)
# path name of trained models
PATH = './model/DeepOPFSB'+method+'_'+kcoef+'E'+str(Epoch)+'.pth'
PATHs = './model/DeepOPFSB'+method+'_'+kcoef
resultnm =  './result/Res_DeepOPFSB'+method+'_'+kcoef+'E'+str(Epoch)+str(T)+'Nt'+str(Ntest)+'.mat'
fignm = './result/ls_DeepOPFSB'+method+'_'+kcoef+'E'+str(Epoch)+'.jpg'
print('fignm', fignm)

print('***************begin case',Nbus,'sys_R',sys_R,'***************')
print('kcoef',kcoef,'\nkcoef',kcoef,'\nPATHs',PATHs,'\nPATH',PATH,'\nresultnm',resultnm)
print('training setting: ','T', T,'Ntrain',Ntrain,'Ntest',Ntest,'Nhis', Nhis,'batch_size',batch_size_training, \
      ' Lr', Lr,' Epoch',Epoch)
print('kcost', kcost,'kpd',kpd_max,'kqd',kqd_max,'kgenp',kgenp_max,'kgenq',kgenq_max,'k_dV',k_dV)

# load data
if Nbus == 30:
    # B30
    matnm = './data/XY_case30_real_Nsample5W_ipopt.mat' 
    matparanm = './data/case30_para.mat'
    matcommunitynm = './data/case30_QLij_community4.mat'
    
elif Nbus == 118:
    # B118
    matnm = './data/XY_case'+str(Nbus)+'r'+str(sys_R)+'real'+'_ipopt_5W'+'.mat'
    matparanm = './data/pglib_opf_case'+str(Nbus)+'_ieeer'+str(sys_R)+'_para.mat'
    # matcommunitynm = './data/pglib_opf_case118_ieeer4_Z_community6.mat'
    matcommunitynm = './data/pglib_opf_case118_ieeer4_SLij_community7.mat'
    

# matnm = './data/XY_case'+str(Nbus)+'r'+str(sys_R)+'real'+'_ipopt_5W'+'.mat'
# matparanm = './data/pglib_opf_case'+str(Nbus)+'_ieeer'+str(sys_R)+'_para.mat' -->
# matcommunitynm = './data/pglib_opf_case118_ieeer4_Z_community6.mat'
# print('matnm',matnm,'\nmatparanm',matparanm,'\nmatcommunitynm',matcommunitynm)

mat = scipy.io.loadmat(matnm)   
matpara = scipy.io.loadmat(matparanm)
# matcommunity = scipy.io.loadmat(matcommunitynm)

device cuda:0
Let's use 1 GPUs!
fignm ./result/ls_DeepOPFSB_krp_B30r0V1S32000Nhis3lr0.0001kc0.0kd100kg2000kv500ky1L32224bs50E4500.jpg
***************begin case 30 sys_R 0 ***************
kcoef B30r0V1S32000Nhis3lr0.0001kc0.0kd100kg2000kv500ky1L32224bs50 
kcoef B30r0V1S32000Nhis3lr0.0001kc0.0kd100kg2000kv500ky1L32224bs50 
PATHs ./model/DeepOPFSB_krp_B30r0V1S32000Nhis3lr0.0001kc0.0kd100kg2000kv500ky1L32224bs50 
PATH ./model/DeepOPFSB_krp_B30r0V1S32000Nhis3lr0.0001kc0.0kd100kg2000kv500ky1L32224bs50E4500.pth 
resultnm ./result/Res_DeepOPFSB_krp_B30r0V1S32000Nhis3lr0.0001kc0.0kd100kg2000kv500ky1L32224bs50E45001Nt10000.mat
training setting:  T 1 Ntrain 32000 Ntest 10000 Nhis 3 batch_size 50  Lr 0.0001  Epoch 4500
kcost tensor([0.0002]) kpd tensor([100.]) kqd tensor([100.]) kgenp tensor([2000.]) kgenq tensor([2000.]) k_dV 0.1


In [3]:
# samples and power system parameters
if Nbus == 30:
    # load 1: with all loads in samples
    RPd = mat['RPd']
    RQd = mat['RQd']
    gen = matpara['gen'][:, [0,3,4,8,9]]
    gencost = matpara['gencost'][:, 4:8]
    branch = matpara['branch'][:, [0,1,5,11,12]]
elif Nbus == 118 or Nbus == 300:
    # load 2: with only nonzero loads in samples
    load_idx = (mat['load_idx'].squeeze() - 1).astype(int)
    print('load_idx',load_idx.shape)
    RPd = np.zeros((mat['RPd'].shape[0], Nbus))
    RQd = np.zeros((mat['RQd'].shape[0], Nbus))
    RPd[:, load_idx] = mat['RPd']
    RQd[:, load_idx] = mat['RQd']
    
    gen = matpara['gen']
    gencost = matpara['gencost']
    branch = matpara['branch']
    
RPg = mat['RPg']
RQg = mat['RQg']
Ybus = matpara['Ybus']
Yf = matpara['Yf']
Yt = matpara['Yt']
bus = matpara['bus']
baseMVA = matpara['baseMVA']
bus_slack = np.squeeze(np.where(bus[:, 1] == 3))
Nbranch = np.size(branch, axis = 0)
Ybus_full = Ybus*1.0
bus_Va = np.delete(np.arange(Nbus),bus_slack)
BRANFT = torch.from_numpy(branch[:, 0:2] - 1).long()
print('bus_Va',bus_Va.shape)
print('bus_slack',bus_slack)
print('Nbus',Nbus,'Nbranch',Nbranch,'maxRPd',mat['RPd'].shape,'maxRQd',mat['RQd'].shape)
print('gencost',gencost[0])

# incidance matricx of branch from
finc = np.zeros((branch.shape[0], Nbus), dtype = float)
tinc = np.zeros((branch.shape[0], Nbus), dtype = float)
for i in range(branch.shape[0]):
    finc[i,  branch[i, 0].astype(int)-1] = 1
    tinc[i,  branch[i, 1].astype(int)-1] = 1

# narrow down the upper and lower bounds of predicted Va Vm
if Nbus == 30:
    VmLb = torch.tensor([0.98])
    VmUb = torch.tensor([1.06])
    VaLb = -math.pi*17/180
    VaUb = -math.pi*4/180
elif Nbus == 118:
    VmLb = torch.tensor([1.02])
    VmUb = torch.tensor([1.06])
    VaLb = -math.pi*20/180
    VaUb = math.pi*16/180
print('VmLb', VmLb, 'VmUb', VmUb, 'VaLb', VaLb, 'VaUb', VaUb)

# spase matrix of admittance matrix: for computation speedup
Ybus = sparse.csr_matrix(Ybus)
Yf = sparse.csr_matrix(Yf) 
Yt = sparse.csr_matrix(Yt)

# sample index for random sampling
idx_sample = random.sample(range(0,RPd.shape[0]),Nsample)
idx_train = np.asarray(idx_sample[0:Ntrain])
idx_train_label = np.asarray(idx_sample[Ntrain: Nhis+Ntrain])
idx_test = np.asarray(idx_sample[-Ntest:])
idx_his = idx_train_label
print('idx_train',idx_train.shape,'idx_train_label',idx_train_label.shape,'idx_test',idx_test.shape)
      
# find bus number of buses with zero and non-zero net power injection: for load satisfaction constraints
bus_Pd = np.squeeze(np.where(np.abs(RPd[0,:]) > 0), axis=0)
bus_Pd0 = np.squeeze(np.where(RPd[0,:] == 0), axis=0)
bus_Qd = np.squeeze(np.where(np.abs(RQd[0,:]) > 0), axis=0)
bus_Qd0 = np.squeeze(np.where(RQd[0,:] == 0), axis=0)
print('bus_Pd',bus_Pd.shape,'bus_Pd0', bus_Pd0.shape, '\nbus_Qd',bus_Qd.shape, 'bus_Qd0', bus_Qd0.shape)

# input data: non-zero Pd and Qd
x = np.concatenate((RPd[:, bus_Pd], RQd[:, bus_Qd]), axis = 1)/baseMVA
print('x', x.shape)

# output data: Vm Va
yvm  = mat['RVm']
yva = mat['RVa']*math.pi/180
print('yvmmin', np.min(yvm), 'yvmmax', np.max(yvm))
print('yvamin', np.min(yva), 'yvamax', np.max(yva))

# historical voltage: for post-porocessing
his_V = np.mean(yvm[idx_his]*np.exp(1j *yva[idx_his]),axis=0)
print('his_V',his_V.shape)

# tensor form
x_tensor = torch.from_numpy(x).float()
yva_tensor = torch.from_numpy(yva).float()
yvm_tensor = torch.from_numpy(yvm).float()
print('x_tensor',x_tensor.shape,'yva_tensor',yva_tensor.shape,'yvm_tensor',yvm_tensor.shape)

# since OPF is global, the input of each group has the global information
# train data
xtrain = x_tensor[idx_train]
yvatrain = yva_tensor[idx_train]             
yvmtrain = yvm_tensor[idx_train]

# test data
xtest = x_tensor[idx_test]
yvatest = yva_tensor[idx_test]
yvmtest = yvm_tensor[idx_test]
print('Ntrain',Ntrain,'xtrain', xtrain.shape,'yvatrain', yvatrain.shape,'yvmtrain', yvmtrain.shape)
print('Ntest',Ntest,'xtest', xtest.shape,'yvatest', yvatest.shape,'yvmtest', yvmtest.shape)
print('yvatrain',torch.min(yvatrain[:, bus_Va]), torch.max(yvatrain[:, bus_Va]))
print('yvmtrain',torch.min(yvmtrain), torch.max(yvmtrain))
print('yvatest',torch.min(yvatest[:, bus_Va]), torch.max(yvatest[:, bus_Va]))
print('yvmtest',torch.min(yvmtest), torch.max(yvmtest))

# delete variables that are not used any more
del x,x_tensor,yvm,yva,yvm_tensor,yva_tensor
gc.collect()

bus_Va (29,)
bus_slack 0
Nbus 30 Nbranch 41 maxRPd (50000, 30) maxRQd (50000, 30)
gencost [0.02 2.   0.  ]
VmLb tensor([0.9800]) VmUb tensor([1.0600]) VaLb -0.29670597283903605 VaUb -0.06981317007977318
idx_train (32000,) idx_train_label (3,) idx_test (10000,)
bus_Pd (20,) bus_Pd0 (10,) 
bus_Qd (20,) bus_Qd0 (10,)
x (50000, 40)
yvmmin 0.95 yvmmax 1.05
yvamin -0.10093294776740376 yvamax 0.0
his_V (30,)
x_tensor torch.Size([50000, 40]) yva_tensor torch.Size([50000, 30]) yvm_tensor torch.Size([50000, 30])
Ntrain 32000 xtrain torch.Size([32000, 40]) yvatrain torch.Size([32000, 30]) yvmtrain torch.Size([32000, 30])
Ntest 10000 xtest torch.Size([10000, 40]) yvatest torch.Size([10000, 30]) yvmtest torch.Size([10000, 30])
yvatrain tensor(-0.1009) tensor(-0.0043)
yvmtrain tensor(0.9500) tensor(1.0500)
yvatest tensor(-0.1009) tensor(-0.0044)
yvmtest tensor(0.9500) tensor(1.0500)


0

In [4]:
# Pg Qg constraints
# find the generation that with positive power output: to improve performance
# some generation may only output P or Q
# find generators really output active/reactive power
idxPg = np.squeeze(np.where(gen[:, 3] > 0), axis=0)
idxQg = np.squeeze(np.where(gen[:, 1] > 0), axis=0)
bus_Pg = gen[idxPg, 0] - 1 # python: bus number start from 0
bus_Qg = gen[idxQg, 0] - 1 # python: bus number start from 0
bus_Pg = bus_Pg.astype(int)
bus_Qg = bus_Qg.astype(int)
print('bus_Pg',bus_Pg.shape, bus_Pg,'\nbus_Qg', bus_Qg.shape,bus_Qg)

# numpy to tensor
bus_nonPg = np.arange(Nbus)
bus_nonPg = torch.from_numpy(np.delete(bus_nonPg,bus_Pg)).long()
bus_nonQg = np.arange(Nbus)
bus_nonQg = torch.from_numpy(np.delete(bus_nonQg,bus_Qg)).long()

# find load buses
Pnet_nonPg = RPd[0,:]*1.0
Pnet_nonQg = RQd[0,:]*1.0
Pnet_nonPg[bus_Pg] = 0
Pnet_nonQg[bus_Qg] = 0
bus_Pnet_nonPg = np.squeeze(np.where(np.abs(Pnet_nonPg) > 0), axis=0)
bus_Pnet_nonQg = np.squeeze(np.where(np.abs(Pnet_nonQg) > 0), axis=0)
print('bus_Pnet_nonPg',bus_Pnet_nonPg.shape,bus_Pnet_nonPg,'\nbus_Pnet_nonQg',bus_Pnet_nonQg.shape,bus_Pnet_nonQg)

# bus number of generators
bus_gen = gen[:,0].astype(int) - 1
print('bus_gen',bus_gen)

# find non-ZIB buses
Pnet = RPd[0,:]*1.0
Pnet[bus_gen] = Pnet[bus_gen] + 10

# bus number of non-ZIBs
bus_Pnet_all = np.squeeze(np.where(np.abs(Pnet) > 0), axis=0)

# bus number of ZIBs
bus_ZIB_all = np.squeeze(np.where(np.abs(Pnet) == 0), axis=0)
print('bus_Pnet_all',bus_Pnet_all.shape,bus_Pnet_all,'\nbus_ZIB_all',bus_ZIB_all.shape,bus_ZIB_all)

# number of ZIBs
NZIB = bus_ZIB_all.shape[0] 
print('NZIB',NZIB)

# index of non-ZIBs
idx_Pnet = np.concatenate((bus_Pnet_all, bus_Pnet_all+Nbus), axis=0)

# index of ZIBs
idx_ZIB = np.concatenate((bus_ZIB_all, bus_ZIB_all+Nbus), axis=0)
print('idx_ZIB', idx_ZIB.shape)

# index of the slack bus in non-ZIBs
idx_bus_Pnet_slack = np.where(bus_Pnet_all == bus_slack)[0]
print('idx_bus_Pnet_slack',idx_bus_Pnet_slack)

bus_Pnet_noslack_all = np.delete(bus_Pnet_all, idx_bus_Pnet_slack, axis=0)
print('bus_Pnet_noslack_all',bus_Pnet_noslack_all)

# generation cost coefficients
gencost_Pg = gencost[idxPg, :]
gencost_tensor = torch.from_numpy(gencost_Pg).float()
# get the upper and lower bounds for generators
MAXMIN_Pg = gen[idxPg, 3:5]/baseMVA
MAXMIN_Qg = gen[idxQg, 1:3]/baseMVA
MAXMIN_Pg_tensor = torch.from_numpy(MAXMIN_Pg).float()
MAXMIN_Qg_tensor = torch.from_numpy(MAXMIN_Qg).float()
print('gencost_Pg',gencost_Pg.shape,'MAXMIN_Pg',MAXMIN_Pg.shape,'MAXMIN_Qg',MAXMIN_Qg.shape)

# test load and generation
Pdtest = RPd[idx_test]/baseMVA
Qdtest = RQd[idx_test]/baseMVA
Pgtest = RPg[idx_test][:, idxPg]/baseMVA
Qgtest = RQg[idx_test][:, idxQg]/baseMVA

bus_Pg (6,) [ 0  1 21 26 22 12] 
bus_Qg (6,) [ 0  1 21 26 22 12]
bus_Pnet_nonPg (18,) [ 2  3  6  7  9 11 13 14 15 16 17 18 19 20 23 25 28 29] 
bus_Pnet_nonQg (18,) [ 2  3  6  7  9 11 13 14 15 16 17 18 19 20 23 25 28 29]
bus_gen [ 0  1 21 26 22 12]
bus_Pnet_all (24,) [ 0  1  2  3  6  7  9 11 12 13 14 15 16 17 18 19 20 21 22 23 25 26 28 29] 
bus_ZIB_all (6,) [ 4  5  8 10 24 27]
NZIB 6
idx_ZIB (12,)
idx_bus_Pnet_slack [0]
bus_Pnet_noslack_all [ 1  2  3  6  7  9 11 12 13 14 15 16 17 18 19 20 21 22 23 25 26 28 29]
gencost_Pg (6, 3) MAXMIN_Pg (6, 2) MAXMIN_Qg (6, 2)


In [5]:
# remove Va of slack bus from the training data
yvtrain_Pnet =  torch.cat((yvatrain[:, bus_Pnet_noslack_all.tolist()], yvmtrain[:, bus_Pnet_all.tolist()]), dim=1)
yvttest_Pnet =  torch.cat((yvatest[:, bus_Pnet_noslack_all.tolist()], yvmtest[:, bus_Pnet_all.tolist()]), dim=1)
print('yvtrain_Pnet',yvtrain_Pnet.shape,'yvttest_Pnet',yvttest_Pnet.shape) 

# training batch data
training_dataset_v = Data.TensorDataset(xtrain, yvtrain_Pnet)
training_loader_v = Data.DataLoader(
        dataset=training_dataset_v,
        batch_size=batch_size_training,
        shuffle=True,
    )   


# test batch data
batch_size_test = 1
test_dataset_v = Data.TensorDataset(xtest, yvttest_Pnet)
test_loader_v = Data.DataLoader(
        dataset=test_dataset_v,
        batch_size=batch_size_test,
        shuffle=False,
    ) 

# number of predicted variables
NPred_Vm = bus_Pnet_all.shape[0]
NPred_Va = bus_Pnet_noslack_all.shape[0]

# scaling and bias of Vm Va
Vmscale = torch.ones(NPred_Vm)*(VmUb - VmLb)
Vmbias = torch.ones(NPred_Vm)*VmLb
print('Vmscale',Vmscale.shape,Vmscale[0],'Vmbias',Vmbias.shape,Vmbias[0])

Vascale = torch.ones(NPred_Va)*(VaUb - VaLb)
Vabias = torch.ones(NPred_Va)*VaLb
print('Vascale',Vascale.shape,Vascale[0],'Vabias',Vabias.shape,Vabias[0])

Vscale = torch.cat((Vascale, Vmscale), dim=0)
Vbias = torch.cat((Vabias, Vmbias), dim=0)
print('Vscale',Vscale.shape,Vscale[0],'Vbias',Vbias.shape,Vbias[0])

yvtrain_Pnet torch.Size([32000, 47]) yvttest_Pnet torch.Size([10000, 47])
Vmscale torch.Size([24]) tensor(0.0800) Vmbias torch.Size([24]) tensor(0.9800)
Vascale torch.Size([23]) tensor(0.2269) Vabias torch.Size([23]) tensor(-0.2967)
Vscale torch.Size([47]) tensor(0.2269) Vbias torch.Size([47]) tensor(-0.2967)


In [6]:
bus_slack

array(0, dtype=int64)

In [7]:
# loss function coefficient
Npg, Nqg = bus_Pg.shape[0], bus_Qg.shape[0]
Npd, Nqd = bus_Pnet_nonPg.shape[0], bus_Pnet_nonQg.shape[0]
if flag_k == 2:
    kpd = torch.ones(Npd)
    kqd = torch.ones(Nqd)
    kgenp = torch.ones(Npg)
    kgenq = torch.ones(Nqg)
    kv = torch.ones(NZIB)
elif flag_k == 1:
    kpd = torch.ones(Npd)*kpdi
    kqd = torch.ones(Nqd)*kqdi
    kgenp = torch.ones(Npg)*kgenpi
    kgenq = torch.ones(Nqg)*kgenqi
    kv= torch.ones(NZIB)*kvi

In [8]:
# parameter to recover ZIB: complex form
Yyy1 = Ybus_full[bus_ZIB_all, :]
Yyy = Yyy1[:, bus_ZIB_all]
Yyx1 = Ybus_full[bus_ZIB_all, :]
Yyx = Yyx1[:, bus_Pnet_all]
param_ZIMV = -np.dot(np.linalg.inv(Yyy),Yyx)
print('param_ZIMV',param_ZIMV.shape)

# parameters for ZIBs when calculating gradients
Yba1 = Ybus_full[bus_ZIB_all]
Yba = Yba1[:, bus_Pnet_all]

Ybb1 = Ybus_full[bus_ZIB_all]
Ybb = Ybb1[:, bus_ZIB_all]

Gba, Bba = np.real(Yba), np.imag(Yba)
Gbb, Bbb = np.real(Ybb), np.imag(Ybb)
Ax_r1 = np.concatenate((Gba, -Bba), axis=1) 
Ax_r2 = np.concatenate((Bba,  Gba), axis=1) 
Ax = np.concatenate((Ax_r1,  Ax_r2), axis=0) 

Ay_r1 = np.concatenate((Gbb, -Bbb), axis=1) 
Ay_r2 = np.concatenate((Bbb,  Gbb), axis=1) 
Ay = np.concatenate((Ay_r1,  Ay_r2), axis=0)

Ay_inv = np.linalg.inv(Ay)
param_ZIM = - np.dot(Ay_inv,Ax)
print('param_ZIM',param_ZIM.shape)

# Jacobian matrix in Cartesian coordinate system
# J[i] = np.dot(diage2, Me) + np.dot(diagf2, Mf) + Mab
Me1 = np.concatenate((Ybus_full.real,  -Ybus_full.imag), axis=1)
Me2 = np.concatenate((-Ybus_full.imag,  -Ybus_full.real), axis=1)
Me = np.concatenate((Me1, Me2), axis=0)

Mf1 = np.concatenate((Ybus_full.imag,  Ybus_full.real), axis=1)
Mf2 = np.concatenate((Ybus_full.real,  -Ybus_full.imag), axis=1)
Mf = np.concatenate((Mf1, Mf2), axis=0)

MGB = np.concatenate((Ybus_full.real, -Ybus_full.imag), axis=1)
MBG = np.concatenate((Ybus_full.imag, Ybus_full.real), axis=1)
print('Me',Me.shape,'Mf',Mf.shape,'MGB',MGB.shape,'MBG',MBG.shape)

# for element-wise product (for batch training)
Me_expd = np.expand_dims(Me, axis=0)
Mf_expd = np.expand_dims(Mf, axis=0)
Me_re = np.repeat(Me_expd, batch_size_training, axis=0)
Mf_re = np.repeat(Mf_expd, batch_size_training, axis=0)
print('Me_expd',Me_expd.shape,'Mf_expd',Mf_expd.shape)
print('Me_re',Me_re.shape,'Mf_re',Mf_re.shape)

MGB_expd = np.expand_dims(MGB, axis=0)
MBG_expd = np.expand_dims(MBG, axis=0)
MGB_re = np.repeat(MGB_expd, batch_size_training, axis=0)
MBG_re = np.repeat(MBG_expd, batch_size_training, axis=0)
print('MGB_expd',MGB_expd.shape,'MGB_re',MGB_re.shape)
print('MBG_expd',MBG_expd.shape,'MBG_re',MBG_re.shape)

MGB_re_tensor = torch.from_numpy(MGB_re)
MBG_re_tensor = torch.from_numpy(MBG_re)
Me_re_tensor = torch.from_numpy(Me_re)
Mf_re_tensor = torch.from_numpy(Mf_re)
param_ZIM_tensor = torch.from_numpy(param_ZIM)
param_ZIM_tensor_expd = torch.unsqueeze(param_ZIM_tensor, dim=0)
param_ZIM_tensor_re = param_ZIM_tensor_expd.repeat_interleave(batch_size_training, dim=0)

param_ZIMV (6, 24)
param_ZIM (12, 48)
Me (60, 60) Mf (60, 60) MGB (30, 60) MBG (30, 60)
Me_expd (1, 60, 60) Mf_expd (1, 60, 60)
Me_re (50, 60, 60) Mf_re (50, 60, 60)
MGB_expd (1, 30, 60) MGB_re (50, 30, 60)
MBG_expd (1, 30, 60) MBG_re (50, 30, 60)


In [9]:
## other function
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

def matrix_diag(x):
    # x是二维数组
    n, d = x.shape
    ret = np.zeros((n, d, d))
    index = (np.arange(0, n * d * d, d * d).reshape(-1, 1) +
          np.arange(0, d * d, d + 1)).reshape(-1)
    ret.flat[index] = x.reshape(-1)
    return ret

def tensor_coo2d(x):
    idx = torch.nonzero(x).T
    data = x[idx[0],idx[1],idx[2]]
    coo_x = torch.sparse_coo_tensor(idx, data, x.shape)
    return coo_x

def get_clamp(Pred, Predmin, Predmax):
    # each row is a sample;Predmin and Predmax is the limit for each element of each row
    Pred_clip = Pred.clamp(min = Predmin, max = Predmax)
    return Pred_clip

# testing Vm
def get_mae(real, predict):
    '''
    mean absolute error
    '''
    if len(real) == len(predict):
        if torch.is_tensor(real):
            err = torch.mean(torch.abs(real - predict)) 
        else:
            err = np.mean(np.abs(real - predict))
            
        return err
    else:
        return None
    
def get_rerr(real, predict):
    '''
    relative error
    '''
    if len(real) == len(predict):
        if torch.is_tensor(real):
            err = torch.abs((predict - real) / real)*100
        else:
            err = np.abs((predict - real) / real)*100
        return err
    else:
        return None
    
def get_rerr2(real, predict):
    '''
    relative error
    '''
    if len(real) == len(predict):       
        err = (predict - real) / real*100          
        return err
    else:
        return None
    
# power balance
def get_PQ(V):
    S = np.zeros(V.shape,dtype = 'complex_')
    for i in range(V.shape[0]):
#         I = np.dot(Ybus, V[i]).conj()
        I = Ybus.dot(V[i]).conj()
        S[i]  = np.multiply(V[i], I)
    
    P = np.real(S)
    Q = np.imag(S) 
    return P, Q
        
def get_genload(V, Pdtest, Qdtest, bus_Pg, bus_Qg):
    S = np.zeros(V.shape,dtype = 'complex_')
    for i in range(V.shape[0]):
#         I = np.dot(Ybus, V[i]).conj()
        I = Ybus.dot(V[i]).conj()
        S[i]  = np.multiply(V[i], I)
    
    P = np.real(S)
    Q = np.imag(S) 

    # bus_Pg = bus_Pg.cpu()
    # bus_Qg = bus_Qg.cpu()
    Pg = P[:, bus_Pg] + Pdtest[:, bus_Pg]
    Qg = Q[:, bus_Qg] + Qdtest[:, bus_Qg]   
    Pd = -P
    Qd = -Q
    Pd[:, bus_Pg] = Pg - P[:, bus_Pg]
    Qd[:, bus_Qg] = Qg - Q[:, bus_Qg]   
    return Pg, Qg, Pd, Qd

def get_Pgcost(Pg,idxPg,gencost):
    cost = (gencost[:, 0]*(Pg*Pg) + gencost[:, 1]*Pg)   # cost = a*Pg^2+b*Pg+c
    return np.sum(cost, 1)

In [10]:
# torch.pow(input,exponent)
class Penalty_V(Function):
    @staticmethod
    def forward(ctx, V, PQd):
        Nsam = V.shape[0]  
        xam_P = np.insert(V.detach().cpu().numpy(), idx_bus_Pnet_slack, 0, axis=1) # insert slack bus fi = 0
        Vx = xam_P[:, NPred_Vm:]*np.exp(1j*xam_P[:, 0:NPred_Vm]) 
        # print('Vm', xam_P[7, NPred_Vm:], '\nVa', xam_P[7, 0:NPred_Vm])
        
        # recover ZIBs Va Vm
        Vy = np.dot(param_ZIMV, Vx.T).T
        # print('Vx',Vx.shape,'Vy',Vy.shape,)
        
        Ve = np.zeros((Nsam, Nbus));
        Vf = np.zeros((Nsam, Nbus));
        Ve[:, bus_Pnet_all] = Vx.real
        Vf[:, bus_Pnet_all] = Vx.imag
        Ve[:, bus_ZIB_all] = Vy.real
        Vf[:, bus_ZIB_all] = Vy.imag   
        Pred_V = Ve + 1j*Vf
        
        PQd = PQd.detach()
        Pdtest = torch.zeros((Nsam, Nbus)).float().to(device) # all buses
        Qdtest = torch.zeros((Nsam, Nbus)).float().to(device) # all buses
        Pdtest[:, bus_Pd] = PQd[:, 0: bus_Pd.shape[0]]# none zero load (may contain generation bus)
        Qdtest[:, bus_Qd] = PQd[:, bus_Pd.shape[0]:] # none zero load (may contain generation bus)

        # S = P + j*Q; P = Pg - Pd; Q = Qg - Qd;
        Pred_S = np.zeros(Ve.shape, dtype = 'complex_') 
        Pred_I = np.zeros(Ve.shape, dtype = 'complex_') 
        for i in range(Nsam):
            Pred_I[i] = Ybus.dot(Pred_V[i]).conj()
            Pred_S[i] = np.multiply(Pred_V[i], Pred_I[i])

        Pred_P = torch.from_numpy(np.real(Pred_S)).float().to(device)
        Pred_Q = torch.from_numpy(np.imag(Pred_S)).float().to(device)
        Pg = Pred_P + Pdtest
        Qg = Pred_Q + Qdtest
        
        # print('Pg[:, bus_Pg]', Pg[0, bus_Pg],'\nQg[:, bus_Pg]', Qg[0, bus_Qg])

        # generator limits  
        loss_Pgi = torch.sum((torch.clamp(Pg[:, bus_Pg] - MAXMIN_Pg_tensor[:, 1], max=0).pow(2) +  \
                   torch.clamp(Pg[:, bus_Pg] - MAXMIN_Pg_tensor[:, 0], min=0).pow(2)), dim = 0)
        
        loss_Qgi = torch.sum((torch.clamp(Qg[:, bus_Qg] - MAXMIN_Qg_tensor[:, 1], max=0).pow(2) + \
                   torch.clamp(Qg[:, bus_Qg] - MAXMIN_Qg_tensor[:, 0], min=0).pow(2)), dim = 0)
        
        mat_PgLB = torch.clamp(Pg[:, bus_Pg] - MAXMIN_Pg_tensor[:, 1], max=0)
        mat_PgUB = torch.clamp(Pg[:, bus_Pg] - MAXMIN_Pg_tensor[:, 0], min=0)
        mat_QgLB = torch.clamp(Qg[:, bus_Qg] - MAXMIN_Qg_tensor[:, 1], max=0)
        mat_QgUB = torch.clamp(Qg[:, bus_Qg] - MAXMIN_Qg_tensor[:, 0], min=0)
        
        # print('mat_PgLB', mat_PgLB[0],'\nmat_PgUB', mat_PgUB[0],'\nmat_QgLB', mat_QgLB[0],'\nmat_QgUB', mat_QgUB[0])
        # print('loss_Pgi', loss_Pgi,'\nloss_Qgi', loss_Qgi)

        loss_Pg = loss_Pgi.sum()   
        loss_Qg = loss_Qgi.sum()
            
        # for non generation bus
        loss_Pdi = torch.sum(Pg[:, bus_Pnet_nonPg].pow(2), dim = 0)
        loss_Qdi = torch.sum(Qg[:, bus_Pnet_nonQg].pow(2), dim = 0)
        
        # loss_Pdi = torch.sum(Pg[:, bus_nonPg].pow(2), dim = 0)
        # loss_Qdi = torch.sum(Qg[:, bus_nonQg].pow(2), dim = 0)
        # print('loss_Pdi', loss_Pdi[0], 'loss_Qdi', loss_Qdi[0])
        # print('loss_ZIB_Pd', Pg[0, bus_ZIB_all], 'loss_ZIB_Qd', Qg[0, bus_ZIB_all])

        loss_Pd = loss_Pdi.sum()
        loss_Qd = loss_Qdi.sum()

        # Pg cost; ignore constant c0        
        absPg = torch.where(Pg[:, bus_Pg] > 0, Pg[:, bus_Pg], -Pg[:, bus_Pg]*2.0)
        # print('absPg',absPg.shape,'gencost_tensor',gencost_tensor.shape)
        loss_Pgcost = (gencost_tensor[:, 0]*torch.pow(Pg[:, bus_Pg], 2) + gencost_tensor[:, 1]*absPg).sum()
        
        # volatge violation of ZIB
        Vm_ZIB = torch.from_numpy(np.sqrt(Ve[:, bus_ZIB_all]**2 + Vf[:, bus_ZIB_all]**2)).to(device)
        loss_Vi = torch.sum((torch.clamp(Vm_ZIB - VmLb[0], max=0).pow(2) +  \
                             torch.clamp(Vm_ZIB - VmUb[0], min=0).pow(2)), dim = 0)
        
        # total loss
        global kcost, kpd, kqd, kgenp, kgenq, kv
        if flag_k == 2:
            # update penalty coefficient 
            kgenp = torch.min(kcost*loss_Pgcost/(loss_Pgi+1e-4), kgenp_max)
            kgenq = torch.min(kcost*loss_Pgcost/(loss_Qgi+1e-4), kgenq_max)
            kpd = torch.min(kcost*loss_Pgcost/(loss_Pdi+1e-4), kpd_max)
            kqd = torch.min(kcost*loss_Pgcost/(loss_Qdi+1e-4), kqd_max)
            kv = torch.min(kcost*loss_Pgcost/(loss_Vi+1e-4), kv_max)
        
        ls_cost  = (kcost*loss_Pgcost)/Nsam
        ls_Pg = (kgenp*loss_Pgi).sum()/Nsam
        ls_Qg = (kgenq*loss_Qgi).sum()/Nsam
        ls_Pd = (kpd*loss_Pdi).sum()/Nsam
        ls_Qd = (kqd*loss_Qdi).sum()/Nsam 
        ls_V = (kv*loss_Vi).sum()/Nsam
    
        loss_out = ls_cost + ls_Pg + ls_Qg + ls_Pd + ls_Qd + ls_V
        # print('ls_cost',ls_cost,'ls_Pg',ls_Pg,'ls_Qg',ls_Qg,'ls_Pd',ls_Pd,'ls_Qd',ls_Qd,'ls_V',ls_V)
        
        if flag_save_ls:           
            ls_costk_log.append(kcost*loss_Pgcost)
            ls_Pgk_log.append(kgenp*loss_Pgi)
            ls_Qgk_log.append(kgenq*loss_Qgi)
            ls_Pdk_log.append(kpd*loss_Pdi)
            ls_Qdk_log.append(kqd*loss_Qdi)
            ls_Vk_log.append(kv*loss_Vi)

            ls_cost_log.append(loss_Pgcost)
            ls_Pg_log.append(loss_Pgi)
            ls_Qg_log.append(loss_Qgi)
            ls_Pd_log.append(loss_Pdi)
            ls_Qd_log.append(loss_Qdi)
            ls_V_log.append(loss_Vi)

            k_Pg_log.append(kgenp)
            k_Qg_log.append(kgenq)
            k_Pd_log.append(kpd)
            k_Qd_log.append(kqd) 
            k_V_log.append(kv)
        
        # save variables used in backward: calculate gradient
        ctx.save_for_backward(Pg, Qg, torch.as_tensor(Ve), torch.as_tensor(Vf),torch.as_tensor(kgenp), torch.as_tensor(kgenq), \
                              torch.as_tensor(kpd), torch.as_tensor(kqd), torch.as_tensor(xam_P)) 
        return loss_out
    
    def backward(ctx, grad_output):
        Pg, Qg, Ve, Vf, kgenp, kgenq, kpd, kqd, xam_P = ctx.saved_tensors
        
        Ve_tensor = Ve.to(device)
        Vf_tensor = Vf.to(device)
        Vef_tensor = torch.cat((Ve_tensor, Vf_tensor), dim=1)
        xam_P_tensor = xam_P.to(device)
        
        Pg, Qg, Ve, Vf = Pg.detach(), Qg.detach(), Ve.detach(), Vf.detach()
        kgenp, kgenq, kpd, kqd = kgenp.detach(), kgenq.detach(), kpd.detach(), kqd.detach()  
        Nsam, Nbus = Ve.shape
        
        #**************************************************************************#
        # dLv/dVm
        Vm_ZIB = torch.sqrt(Ve[:, bus_ZIB_all]**2 + Vf[:, bus_ZIB_all]**2).to(device)
        # Voltage violation: d(Vm_hat - VmUb)dVef;d(Vm_hat - VmLb)dVef
        mat_Vmax = torch.where(Vm_ZIB - VmUb[0] >  kdelta, ((Vm_ZIB - VmUb[0])/Vm_ZIB), torch.tensor([0.0]).double().to(device))
        mat_Vmin = torch.where(Vm_ZIB - VmLb[0] < -kdelta, ((Vm_ZIB - VmLb[0])/Vm_ZIB), torch.tensor([0.0]).double().to(device))
        mat_V = (mat_Vmin + mat_Vmax)*kv 
        mat_V2 = torch.cat((mat_V, mat_V),dim = 1)   
        # print('Vm_ZIB', Vm_ZIB.shape,'mat_Vmin', mat_Vmin.shape,'mat_Vmax', mat_Vmax.shape)
        # print('mat_V2', mat_V2.shape)
        # print('Vm_ZIB',Vm_ZIB[0],'\nmat_Vmin',mat_Vmin[0],'\nmat_Vmax',mat_Vmax[0])
        
        #**************************************************************************#
        # dLPgQgPdQd/dPQ
        mat_P = torch.zeros(Nsam, Nbus).to(device)
        mat_Q = torch.zeros(Nsam, Nbus).to(device) 
        
        # symbal of Pg: Relu(Pg_hat - Pg_max) + Relu(Pg_min - Pg_hat)
        # loss_Pg.sum()
        #元素小于0，取1；否则取0
        mat_Pgmin = torch.where(Pg[:, bus_Pg] - MAXMIN_Pg_tensor[:, 1] < -kdelta, 2*(Pg[:, bus_Pg] - MAXMIN_Pg_tensor[:, 1]).to(device), torch.tensor([0.0]).to(device))
        #元素大于0，取1；否则取0
        mat_Pgmax = torch.where(Pg[:, bus_Pg] - MAXMIN_Pg_tensor[:, 0] > kdelta, 2*(Pg[:, bus_Pg] - MAXMIN_Pg_tensor[:, 0]).to(device), torch.tensor([0.0]).to(device)) 
        # print('mat_Pgmin', mat_Pgmin.shape, mat_Pgmin[0], 'mat_Pgmax', mat_Pgmax.shape, mat_Pgmax[0],)

        # Pg cost
        mat_Pgneg = torch.where(Pg[:, bus_Pg] > 0, torch.tensor([1.0]).to(device), torch.tensor([-2.0]).to(device))  
        mat_Pgcost = ((2*gencost_tensor[:, 0]).repeat(Nsam, 1)*Pg[:, bus_Pg] + gencost_tensor[:, 1].repeat(Nsam, 1)*mat_Pgneg)  
        # print('mat_Pgneg', mat_Pgcost.shape, mat_Pgneg[0])
            
        mat_P[:, bus_Pg] = (mat_Pgmin+mat_Pgmax)*kgenp.reshape(1, -1) + mat_Pgcost*kcost
     
        # loss_Qg.sum()
        #元素小于0，取1；否则取0
        mat_Qgmin = torch.where(Qg[:, bus_Qg] - MAXMIN_Qg_tensor[:, 1] < -kdelta, 2*(Qg[:, bus_Qg] - MAXMIN_Qg_tensor[:, 1]).to(device), torch.tensor([0.0]).to(device)) 
        #元素大于0，取1；否则取0
        mat_Qgmax = torch.where(Qg[:, bus_Qg] - MAXMIN_Qg_tensor[:, 0] > kdelta, 2*(Qg[:, bus_Qg] - MAXMIN_Qg_tensor[:, 0]).to(device), torch.tensor([0.0]).to(device))       
        mat_Q[:, bus_Qg] = (mat_Qgmin + mat_Qgmax)*kgenq.reshape(1, -1)
        # print('mat_Qgmin', mat_Qgmin.shape, 'mat_Qgmax', mat_Qgmax.shape)
        # print('kgenq.reshape(1, -1)', kgenq.reshape(1, -1))

        # loss_PQd
        mat_P[:, bus_Pnet_nonPg] = (torch.where(torch.abs(Pg[:, bus_Pnet_nonPg]) > kdelta, 2*Pg[:, bus_Pnet_nonPg].to(device), torch.tensor([0.0]).to(device)))*kpd.reshape(1, -1) # matrix with 1 or -1
        mat_Q[:, bus_Pnet_nonQg] = (torch.where(torch.abs(Qg[:, bus_Pnet_nonQg]) > kdelta, 2*Qg[:, bus_Pnet_nonQg].to(device), torch.tensor([0.0]).to(device)))*kqd.reshape(1, -1) # matrix with 1 or -1
        # mat_P[:, bus_nonPg] = (torch.where(torch.abs(Pg[:, bus_nonPg]) > kdelta, 2*Pg[:, bus_nonPg].to(device), torch.tensor([0.0]).to(device)))*kpd.reshape(1, -1) # matrix with 1 or -1
        # mat_Q[:, bus_nonQg] = (torch.where(torch.abs(Qg[:, bus_nonQg]) > kdelta, 2*Qg[:, bus_nonQg].to(device), torch.tensor([0.0]).to(device)))*kqd.reshape(1, -1) # matrix with 1 or -1
        # print('mat_P[:, bus_nonPg]', mat_P[:, bus_nonPg].shape, 'mat_Q[:, bus_nonQg]', mat_Q[:, bus_nonQg].shape, "kpd.reshape(1, -1)", (kpd.reshape(1, -1)).shape)
        
        # dPdVm
        mat_P3 = torch.unsqueeze(mat_P[:, bus_Pnet_all], 2)
        mat_Q3 = torch.unsqueeze(mat_Q[:, bus_Pnet_all], 2)
        # print('mat_P3',mat_P3.shape,'mat_Q3',mat_Q3.shape)
  
        mat_PQ3 = torch.cat((mat_P3, mat_Q3), dim=1)
        # print('mat_PQ3', mat_PQ3.shape)
 
        #*****************************Jacobian matrix***********************************# 
        # start_time = time.process_time()
        diage_expd_tensor = torch.unsqueeze(torch.cat((Ve_tensor, Ve_tensor), dim=1), dim=2)
        diagf_expd_tensor = torch.unsqueeze(torch.cat((Vf_tensor, Vf_tensor), dim=1), dim=2)
        # print('diage_expd_tensor',diage_expd_tensor.shape,'diagf_expd_tensor',diagf_expd_tensor.shape)
        # print('Ve_tensor',Ve_tensor.device,'diagf_expd_tensor',diagf_expd_tensor.device)
        
        diage_re_tensor = diage_expd_tensor.repeat_interleave(2*Nbus, dim=2)
        diagf_re_tensor = diagf_expd_tensor.repeat_interleave(2*Nbus, dim=2)
 
        Vef_expd_tensor = torch.unsqueeze(Vef_tensor, dim=1)
        Vef_re_tensor = Vef_expd_tensor.repeat_interleave(Nbus, dim=1)
        # print('Vef_re_tensor',Vef_re_tensor.device)
        
        a_tensor = torch.sum(MGB_re_tensor.to(device)*Vef_re_tensor, dim=2)
        b_tensor = torch.sum(MBG_re_tensor.to(device)*Vef_re_tensor, dim=2)
        a_diag_tensor = torch.as_tensor(matrix_diag(a_tensor.cpu().numpy()))
        b_diag_tensor = torch.as_tensor(matrix_diag(b_tensor.cpu().numpy()))
        
        Mab_diag1_tensor = torch.cat((a_diag_tensor.to(device), b_diag_tensor.to(device)), dim=2)
        Mab_diag2_tensor = torch.cat((-b_diag_tensor.to(device), a_diag_tensor.to(device)), dim=2)
        Mab_diag_tensor = torch.cat((Mab_diag1_tensor, Mab_diag2_tensor), dim=1)

        # Jacobian matrix
        J_tensor = diage_re_tensor*Me_re_tensor.to(device) + diagf_re_tensor*Mf_re_tensor.to(device) + Mab_diag_tensor
        # Jacobian matrix for non-ZIBs: dPQ_Pnet/dVef_Pnet
        Jx1_tensor = J_tensor[:, idx_Pnet, :]
        Jx_tensor = Jx1_tensor[:, :, idx_Pnet]
        # print('Jx', Jx.shape)
        
        # Jacobian matrix for ZIBs: dPQ_Pnet/dVef_ZIB
        Jy1_tensor = J_tensor[:, idx_Pnet, :]
        Jy_tensor = Jy1_tensor[:, :, idx_ZIB]
        # print('Jy', Jy.shape,'param_ZIM_tensor_re', param_ZIM_tensor_re.shape)

        # dPQ_Pnet/dVef_ZIB*dVef_ZIB/Vef_Pnet
        Jyx_tensor = torch.bmm(Jy_tensor, param_ZIM_tensor_re.to(device))
        
        # dPQ_Pnet/dVef_Pnet: non-ZIBs and ZIBs
        Jcom_tensor = Jx_tensor + Jyx_tensor
        # print('Jcom', Jcom.shape)
        
        #**************************************************************************#
        # dVef_Pnet/dVam_Pnet: from Cartesian coordinate system to polar coordinates
        Vax_tensor, Vmx_tensor = xam_P_tensor[:, 0:NPred_Vm], xam_P_tensor[:, NPred_Vm:]
        # print('Vax', Vax.shape,'Vmx', Vmx.shape)

        # dPQ/dVef
        dPQdVe_tensor = Jcom_tensor[:, :, 0: NPred_Vm];
        dPQdVf_tensor = Jcom_tensor[:, :, NPred_Vm:];
        # print('dPQdVe', dPQdVe.shape,'dPQdVf', dPQdVf.shape)
        
        dPQdVe2_tensor = torch.cat((dPQdVe_tensor, dPQdVe_tensor), dim = 2)
        dPQdVf2_tensor = torch.cat((dPQdVf_tensor, dPQdVf_tensor), dim = 2)
        # print('dPQdVf2', dPQdVf2.shape,'dPQdVe2', dPQdVe2.shape)
 
        # dVef/dVam
        dVedVa_tensor = -Vmx_tensor*torch.sin(Vax_tensor) 
        dVedVm_tensor = torch.cos(Vax_tensor)
        # print('dVedVm', dVedVm.shape,'dVedVa', dVedVa.shape)
        dVfdVa_tensor = Vmx_tensor*torch.cos(Vax_tensor)
        dVfdVm_tensor = torch.sin(Vax_tensor)
        
        dVedVa_expd_tensor = torch.unsqueeze(dVedVa_tensor, dim=1)
        dVedVm_expd_tensor = torch.unsqueeze(dVedVm_tensor, dim=1)
        # print('dVedVm_expd', dVedVm_expd.shape,'dVedVa_expd', dVedVa_expd.shape)
        dVfdVa_expd_tensor = torch.unsqueeze(dVfdVa_tensor, dim=1)
        dVfdVm_expd_tensor = torch.unsqueeze(dVfdVm_tensor, dim=1)

        dVedVa_rep_tensor = dVedVa_expd_tensor.repeat_interleave(NPred_Vm, dim=1)
        dVedVm_rep_tensor = dVedVm_expd_tensor.repeat_interleave(NPred_Vm, dim=1)
        # print('dVedVm_rep', dVedVm_rep.shape,'dVedVa_rep', dVedVa_rep.shape)
        dVfdVa_rep_tensor = dVfdVa_expd_tensor.repeat_interleave(NPred_Vm, dim=1)
        dVfdVm_rep_tensor = dVfdVm_expd_tensor.repeat_interleave(NPred_Vm, dim=1)
   
        dVedVam_tensor = torch.cat((dVedVa_rep_tensor, dVedVm_rep_tensor), dim = 2)
        dVfdVam_tensor = torch.cat((dVfdVa_rep_tensor, dVfdVm_rep_tensor), dim = 2)
        dVe2dVam_tensor = torch.cat((dVedVam_tensor, dVedVam_tensor), dim = 1)
        dVf2dVam_tensor = torch.cat((dVfdVam_tensor, dVfdVam_tensor), dim = 1)
        # print('dVedVam', dVedVam.shape,'dVe2dVam', dVe2dVam.shape)
        
        # dPQd/dVef*dVefx/dVamx: for Pg Qg Pd Qd constaint violation
        Jcom_pole_tensor = dPQdVe2_tensor*dVe2dVam_tensor + dPQdVf2_tensor*dVf2dVam_tensor
        J_slack_tensor = torch.cat((Jcom_pole_tensor[:, :, :idx_bus_Pnet_slack[0]], Jcom_pole_tensor[:, :, idx_bus_Pnet_slack[0]+1:]), dim = 2)
        #**************************************************************************#
        # dLv/dVam: for voltage violaton
        Vefy_tensor = Vef_tensor[:, idx_ZIB]
        dLdVefy_tensor = Vefy_tensor*mat_V2
        # print('dLdVefy', dLdVefy.shape)
        dLdVefy_expd_tensor = torch.unsqueeze(dLdVefy_tensor, dim=2)
        dLdVefy_re_tensor = dLdVefy_expd_tensor.repeat_interleave(2*NPred_Vm, dim=2)
        dLdVefx_tensor = dLdVefy_re_tensor*param_ZIM_tensor_re.to(device)
        # print('dLdVefx_tensor', dLdVefx_tensor.shape,'dLdVefy_re_tensor', dLdVefy_re_tensor.shape,'param_ZIM_tensor_re', param_ZIM_tensor_re.shape)
        dLdVex2_tensor = torch.cat((dLdVefx_tensor[:, :, 0:NPred_Vm], dLdVefx_tensor[:, :, 0:NPred_Vm]), dim = 2)
        dLdVfx2_tensor = torch.cat((dLdVefx_tensor[:, :, NPred_Vm:], dLdVefx_tensor[:, :, NPred_Vm:]), dim = 2)
        # print('dLdVex2', dLdVex2.shape,'dLdVfx2', dLdVfx2.shape)
        dLdVefx2_tensor = torch.cat((dLdVex2_tensor, dLdVfx2_tensor), dim = 1)
        # print('dLdVefx2', dLdVefx2.shape)

        dVedVam_expd_tensor = torch.cat((dVedVa_expd_tensor, dVedVm_expd_tensor), dim = 2)
        dVfdVam_expd_tensor = torch.cat((dVfdVa_expd_tensor, dVfdVm_expd_tensor), dim = 2)
        dVedVamy_re_tensor = dVedVam_expd_tensor.repeat_interleave(2*NZIB, dim=1)
        dVfdVamy_re_tensor = dVfdVam_expd_tensor.repeat_interleave(2*NZIB, dim=1)
        dVefdVamy_tensor = torch.cat((dVedVamy_re_tensor, dVfdVamy_re_tensor), dim = 1)
        # print('dVedVam_expd', dVedVam_expd.shape,'dVfdVam_expd', dVfdVam_expd.shape)
        # print('dVedVamy_re', dVedVamy_re.shape,'dVfdVamy_re', dVfdVamy_re.shape)
        # print('dVefdVamy', dVefdVamy.shape)
        
        dLdVamx_tensor = dLdVefx2_tensor*dVefdVamy_tensor
        dLdVamx_slack_tensor = torch.cat((dLdVamx_tensor[:, :, :idx_bus_Pnet_slack[0]], dLdVamx_tensor[:, :, idx_bus_Pnet_slack[0]+1:]), dim = 2)
        # print('dLdVamx', dLdVamx.shape,'dLdVamx_slack', dLdVamx_slack.shape)
  
        matJ_tensor = mat_PQ3*J_slack_tensor
        # grad_Vloss_unsort = torch.sum(matJ_tensor, dim = 1) + torch.sum(dLdVamx_slack_tensor, dim=1)
        # grad_Vloss = grad_Vloss_unsort[:, idx_bus_Pnet_sort.tolist()]
        
        grad_Vloss = torch.sum(matJ_tensor, dim = 1) + torch.sum(dLdVamx_slack_tensor, dim=1)
        
        return grad_output.to(device)*grad_Vloss.to(device), None 

In [11]:
class NetV(nn.Module):
    def __init__(self, input_channels, output_channels, hidden_units,khidden,Vscale,Vbias):
        super(NetV, self).__init__()
        self.num_layer = khidden.shape[0]
        self.fc1 = nn.Linear(input_channels, khidden[0]*hidden_units)
        self.scale = Vscale.to(device)
        self.bias = Vbias.to(device)
        
        if self.num_layer >= 2: 
            self.fc2 = nn.Linear(khidden[0]*hidden_units, khidden[1]*hidden_units)
        
        if self.num_layer >= 3:
            self.fc3 = nn.Linear(khidden[1]*hidden_units, khidden[2]*hidden_units)
            
            
        self.fcbfend = nn.Linear(khidden[khidden.shape[0]-1]*hidden_units, output_channels)
        self.fcend = nn.Linear(output_channels, output_channels)
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))    
        if self.num_layer >= 2:
            x = F.relu(self.fc2(x))
            
        if self.num_layer >= 3:
            x = F.relu(self.fc3(x))
        
        # fixed final two layers
        x = F.relu(self.fcbfend(x))
        
        x = self.fcend(x)  
        x_PredV = (torch.sigmoid(x)*self.scale + self.bias)
    
        return x_PredV

In [12]:
# neural setting
input_channels = xtrain.shape[1]
MSE_loss = nn.MSELoss()
func_loss = Penalty_V()
output_channels = NPred_Va + NPred_Vm
# train model if it is not test
if flag_test == 0:
    time_trian = 0
    loss_log = []
    # initialize model
    model = NetV(input_channels, output_channels, hidden_units, khidden, Vscale, Vbias)
    # initialize optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=Lr)

    # load model to GPU 
    if torch.cuda.is_available():
        model.to(device)
        print('model.to(device)')
        
        MAXMIN_Pg_tensor = MAXMIN_Pg_tensor.to(device)
        MAXMIN_Qg_tensor = MAXMIN_Qg_tensor.to(device)
        
        # rev
        bus_Pg = torch.tensor(bus_Pg).to(device)        
        bus_Qg = torch.tensor(bus_Qg).to(device)
        bus_Pd = torch.tensor(bus_Pd).to(device)
        bus_Qd = torch.tensor(bus_Qd).to(device)
        bus_nonPg = torch.tensor(bus_nonPg).to(device)
        bus_nonQg = torch.tensor(bus_nonQg).to(device)

        # gencost_tensor, idxPg_tensor, baseMVA_tensor = gencost_tensor.to(device), idxPg_tensor.to(device), baseMVA_tensor.to(device)
        # rev
        gencost_tensor = gencost_tensor.to(device)
        kgenp_max, kgenq_max, kpd_max, kqd_max = kgenp_max.to(device), kgenq_max.to(device), kpd_max.to(device), kqd_max.to(device)
        kcost, kpd, kqd, kgenp, kgenq = kcost.to(device), kpd.to(device), kqd.to(device), kgenp.to(device), kgenq.to(device)
        kv, kv_max = kv.to(device),kv_max.to(device)
        VmUb, VmLb = VmUb.to(device), VmLb.to(device)
        
    

    print('*' * 5+'training'+'*' * 5)
    #Training process: Voltage magnitude
    start_time = time.process_time()
    for epoch in range(Epoch):
        running_loss = 0.0
        for step, (train_x, train_y) in enumerate(training_loader_v):
            #feedforward
            train_x, train_y = train_x.to(device), train_y.to(device)
            yvtrain_hat = model(train_x)   
            #  supervised learning 
            loss = MSE_loss(train_y, yvtrain_hat) # with no penalty
            # loss = kpy*MSE_loss(train_y, yvtrain_hat) + func_loss.apply(yvtrain_hat, train_x)  # with penalty
            
            # # unsupervised learning
            # loss = func_loss.apply(yvtrain_hat, train_x)  # with penalty
            
            running_loss =  running_loss + loss.item()
            # print('train_x',train_x.shape,'train_y', train_y.shape,'yvtrain_hat', yvtrain_hat.shape)

            # backproprogate
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()      

        loss_log.append(running_loss/len(training_loader_v))

        if (epoch+1)%p_epoch == 0:          
#             print('epoch', epoch+1, round(running_loss,4)/len(training_loader_v), \
#                   'Vm',torch.min(yvtrain_hat[:, NPred_Va:]).detach(),torch.max(yvtrain_hat[:, NPred_Va:]).detach(), \
#                   'real',torch.min(train_y[:, NPred_Va:]).detach(),torch.max(train_y[:, NPred_Va:]).detach(), \
#                   'mae', torch.mean(torch.abs(train_y[:, NPred_Va:]-yvtrain_hat[:,NPred_Va:])).detach(), \
#                   'Va',torch.min(yvtrain_hat[:, 0:NPred_Va]).detach(),torch.max(yvtrain_hat[:, 0:NPred_Va]).detach(), \
#                   'real',torch.min(train_y[:, 0:NPred_Va]).detach(),torch.max(train_y[:, 0:NPred_Va]).detach(), \
#                   'mae', torch.mean(torch.abs(train_y[:, 0:NPred_Va]-yvtrain_hat[:, 0:NPred_Va])).detach())    
            
            print('epoch', epoch+1, running_loss/len(training_loader_v), \
                  'Va',torch.min(yvtrain_hat[:, 0:NPred_Va]).detach(),torch.max(yvtrain_hat[:, 0:NPred_Va]).detach(), \
                  'Vm',torch.min(yvtrain_hat[:, NPred_Va:]).detach(),torch.max(yvtrain_hat[:, NPred_Va:]).detach())
            
            print('epoch', epoch+1, 'kcost', kcost, 'kgenp', torch.mean(kgenp), 'kgenq', torch.mean(kgenq), \
                  'kpd', torch.mean(kpd), 'kqd', torch.mean(kqd),'kv', torch.mean(kv),'\n')

        if (epoch+1)%100 == 0 and (epoch+1) >= s_epoch:
            # save trianed model
            torch.save(model.state_dict(), PATHs+'E'+str(epoch+1)+'.pth',_use_new_zipfile_serialization=False)

    time_trian = time.process_time() - start_time  
    print("\n")        
    print('time_trian',np.round(time_trian,5),'seconds')
    print("\n")

    # save trianed model
    torch.save(model.state_dict(), PATH+'.pth',_use_new_zipfile_serialization=False)
        
    loss_log = np.array(loss_log)

    fig = plt.figure(figsize=(15, 6))
    plt.subplot(2,2,1)
    plt.plot(loss_log)
    plt.title('loss')
    plt.subplot(2,2,2)
    plt.plot(loss_log[Epoch-10:])
    plt.title('loss') 
    plt.savefig(fignm)

In [13]:
torch.cuda.empty_cache()
# testing Va using training data
if flag_test: 
    # load trained model 
    print('load trained model: model_v_load')
    model = NetV(input_channels, output_channels, hidden_units, khidden, Vscale, Vbias)
    model.load_state_dict(torch.load(PATH, map_location=device))
    model.eval()
    model.to(device)
    
delta_time = 0
prediction = torch.zeros((Ntest, NPred_Vm+NPred_Va))  
yvmtest_hat = torch.zeros((Ntest, Nbus))
yvatest_hat = torch.zeros((Ntest, Nbus))
for step,(test_x, test_y) in enumerate(test_loader_v):
    if (step+1) % 1 == 0:
        # print('test sample',step+1)   
        test_x = test_x.to(device)
        start_time = time.process_time()
        prediction[step] = model(test_x).detach().cpu()
        delta_time += time.process_time() - start_time


xam_P = torch.from_numpy(np.insert(prediction.numpy(), idx_bus_Pnet_slack, 0, axis=1)) # insert slack bus fi = 0
yvatest_hat[:, bus_Pnet_all.tolist()] = xam_P[:, 0:NPred_Vm]
yvmtest_hat[:, bus_Pnet_all.tolist()] = xam_P[:, NPred_Vm:]

time_NN = delta_time

print('yvmtest',yvmtest[0])
print('yvatest',yvatest[0])
print('before Kron reduction')
print('yvmtest_hat',yvmtest_hat[0])
print('yvatest_hat',yvatest_hat[0])

# recover ZIB
Vetest_ZIB_hat = np.zeros((Ntest, bus_ZIB_all.shape[0]))
Vftest_ZIB_hat = np.zeros((Ntest, bus_ZIB_all.shape[0]))
delta_time = 0
Vx_hat = yvmtest_hat[:, bus_Pnet_all.tolist()].numpy()*np.exp(1j*yvatest_hat[:, bus_Pnet_all.tolist()].numpy())
for i in range(Ntest):
    start_time = time.process_time()
    Vy_hat = np.dot(param_ZIMV, Vx_hat[i])
    Vetest_ZIB_hat[i] = Vy_hat.real
    Vftest_ZIB_hat[i] = Vy_hat.imag
    delta_time += time.process_time() - start_time

time_ZIB = delta_time
Vmy, Vay = cart2pol(Vetest_ZIB_hat, Vftest_ZIB_hat)   
yvmtest_hat[:, bus_ZIB_all] = torch.from_numpy(Vmy).float()
yvatest_hat[:, bus_ZIB_all] = torch.from_numpy(Vay).float()
yvmtest_hat[:, bus_ZIB_all] = get_clamp(yvmtest_hat[:, bus_ZIB_all], VmLb[0].item(), VmUb[0].item())

print('after Kron reduction')
print('yvmtest_hat',yvmtest_hat[0])
print('yvatest_hat',yvatest_hat[0])

mae_Vmtest = get_mae(yvmtest, yvmtest_hat)
mre_Vmtest = get_rerr(yvmtest,yvmtest_hat)
print('mae_Vmtest', mae_Vmtest)
print('mre_Vmtest', torch.mean(mre_Vmtest),'%  ',  torch.max(mre_Vmtest),'%')

mae_Vatest = get_mae(yvatest[:,bus_Va], yvatest_hat[:,bus_Va])
mre_Vatest = get_rerr(yvatest[:,bus_Va],yvatest_hat[:,bus_Va])
print('mae_Vatest', mae_Vatest)
print('mre_Vatest', torch.mean(mre_Vatest),'%  ',  torch.max(mre_Vatest),'%')

load trained model: model_v_load
yvmtest tensor([1.0500, 1.0467, 1.0327, 1.0295, 1.0331, 1.0242, 1.0206, 1.0147, 1.0272,
        1.0289, 1.0272, 1.0333, 1.0500, 1.0254, 1.0278, 1.0258, 1.0238, 1.0182,
        1.0152, 1.0179, 1.0337, 1.0382, 1.0406, 1.0315, 1.0386, 1.0243, 1.0500,
        1.0258, 1.0344, 1.0252])
yvatest tensor([ 0.0000, -0.0106, -0.0310, -0.0369, -0.0335, -0.0429, -0.0461, -0.0490,
        -0.0578, -0.0656, -0.0578, -0.0621, -0.0460, -0.0705, -0.0686, -0.0677,
        -0.0691, -0.0777, -0.0805, -0.0775, -0.0654, -0.0637, -0.0577, -0.0608,
        -0.0473, -0.0533, -0.0352, -0.0443, -0.0519, -0.0637])
before Kron reduction
yvmtest_hat tensor([1.0500, 1.0467, 1.0327, 1.0294, 0.0000, 0.0000, 1.0205, 1.0146, 0.0000,
        1.0288, 0.0000, 1.0332, 1.0500, 1.0254, 1.0278, 1.0257, 1.0238, 1.0182,
        1.0152, 1.0179, 1.0336, 1.0382, 1.0405, 1.0315, 0.0000, 1.0246, 1.0500,
        0.0000, 1.0343, 1.0252])
yvatest_hat tensor([ 0.0000, -0.0698, -0.0698, -0.0698,  0.0000,  0.

In [14]:
## violation calculation function
# Pg Qg violation
def get_vioPQg(Pred_Pg, bus_Pg, MAXMIN_Pg, Pred_Qg, bus_Qg, MAXMIN_Qg):
    vio_PQgmaxminnum = torch.zeros((Pred_Pg.shape[0],4))
    vio_PQgmaxmin = torch.zeros((Pred_Pg.shape[0],4))
    vio_PQg = torch.zeros((Pred_Pg.shape[0],2))
    lsPg = list()
    lsQg = list()
    lsidxPg = np.zeros((Pred_Pg.shape[0]),dtype = np.int32) # index of sample that has violation
    lsidxQg = np.zeros((Pred_Pg.shape[0]),dtype = np.int32) # index of sample that has violation
    kP = 1# inital=0: in order to find nonzero samples 
    kQ = 1
    deltaPgL = np.array([[0, 0]])
    deltaPgU = np.array([[0, 0]])
    deltaQgL = np.array([[0, 0]])
    deltaQgU = np.array([[0, 0]])
    for i in range(Pred_Pg.shape[0]):
        # P
        delta = Pred_Pg[i] - MAXMIN_Pg[:, 0]
        idxPgUB = np.array(np.where(delta > DELTA))
        if np.size(idxPgUB) > 0:
            PgUB = np.concatenate((idxPgUB,delta[idxPgUB]),axis=0).T 
            deltaPgU = np.append(deltaPgU, PgUB, axis = 0)
#             print('idxPgUB',idxPgUB,Pred_Pg[i,idxPgUB],MAXMIN_Pg[idxPgUB, 0],delta[idxPgUB])

        delta = Pred_Pg[i] - MAXMIN_Pg[:, 1]
        idxPgLB = np.array(np.where(delta < -DELTA))
        if np.size(idxPgLB) > 0:
            PgLB = np.concatenate((idxPgLB,delta[idxPgLB]),axis=0).T
            deltaPgL = np.append(deltaPgL, PgLB, axis = 0)
        
        if np.size(idxPgUB)>0 and np.size(idxPgLB) > 0:
            PgLUB = np.concatenate((PgUB,PgLB),axis=0)
        elif np.size(idxPgUB) > 0:
            PgLUB = PgUB
        elif np.size(idxPgLB) > 0:
            PgLUB = PgLB
        
        if (np.size(idxPgUB) + np.size(idxPgLB)) > 0:
            PgLUB = PgLUB[PgLUB[:,0].argsort()]
            lsPg.append(PgLUB)
            lsidxPg[i] = kP # has violation:index of lsPg 
            kP+= 1#index=lsPg

        # Q
        delta = Pred_Qg[i] - MAXMIN_Qg[:, 0]
        idxQgUB = np.array(np.where(delta > DELTA))
        if np.size(idxQgUB) > 0:
            QgUB = np.concatenate((idxQgUB,delta[idxQgUB]),axis=0).T
            deltaQgU = np.append(deltaQgU, QgUB, axis = 0)

        delta = Pred_Qg[i] - MAXMIN_Qg[:, 1]
        idxQgLB = np.array(np.where(delta < -DELTA))
        if np.size(idxQgLB) > 0:
            QgLB = np.concatenate((idxQgLB,delta[idxQgLB]),axis=0).T
            deltaQgL = np.append(deltaQgL, QgLB, axis = 0)
            
        if np.size(idxQgUB) > 0 and np.size(idxQgLB) > 0:   
            QgLUB = np.concatenate((QgUB,QgLB),axis=0)
        elif np.size(idxQgUB) > 0:
            QgLUB = QgUB
        elif np.size(idxQgLB) > 0:
            QgLUB = QgLB
         
        if (np.size(idxQgUB) + np.size(idxQgLB)) > 0:
            QgLUB = QgLUB[QgLUB[:,0].argsort()]
            lsQg.append(QgLUB)
            lsidxQg[i] = kQ # has violation
            kQ+= 1
                    
        
        vio_PQgmaxminnum[i,0] = np.size(idxPgUB)
        vio_PQgmaxminnum[i,1] = np.size(idxPgLB)
        vio_PQgmaxminnum[i,2] = np.size(idxQgUB)
        vio_PQgmaxminnum[i,3] = np.size(idxQgLB)
        
    # Pg Qg violation ratio
    vio_PQgmaxmin[:,0] = (1 - vio_PQgmaxminnum[:, 0]/bus_Pg.shape[0])*100
    vio_PQgmaxmin[:,1] = (1 - vio_PQgmaxminnum[:, 1]/bus_Pg.shape[0])*100
    vio_PQgmaxmin[:,2] = (1 - vio_PQgmaxminnum[:, 2]/bus_Qg.shape[0])*100
    vio_PQgmaxmin[:,3] = (1 - vio_PQgmaxminnum[:, 3]/bus_Qg.shape[0])*100
    vio_PQg[:, 0] = (1 - (vio_PQgmaxminnum[:, 0] + vio_PQgmaxminnum[:, 1])/bus_Pg.shape[0])*100
    vio_PQg[:, 1] = (1 - (vio_PQgmaxminnum[:, 2] + vio_PQgmaxminnum[:, 3])/bus_Qg.shape[0])*100
     
    # delete initial 
    if deltaPgL.shape[0] > 1:
        deltaPgL = np.delete(deltaPgL, 0, axis = 0)
        
    if deltaPgU.shape[0] > 1:    
        deltaPgU = np.delete(deltaPgU, 0, axis = 0)
    
    if deltaQgL.shape[0] > 1:
        deltaQgL = np.delete(deltaQgL, 0, axis = 0)
    
    if deltaQgU.shape[0] > 1:
        deltaQgU = np.delete(deltaQgU, 0, axis = 0)
    
    return lsPg,lsQg,lsidxPg,lsidxQg,vio_PQgmaxmin,vio_PQg, deltaPgL,deltaPgU,deltaQgL,deltaQgU


# branch constraints with violation details for dV
def get_viobran2(Pred_V,Pred_Va,branch,Yf,Yt):
   #branch=[branch from bus, branch to bus, branch limit, minang, maxang]
    branlp = branch[:, 2]/baseMVA
    angminmax = branch[:, 3:5]*math.pi/180
    Pred_branang = Pred_Va[:, BRANFT[:, 0]] - Pred_Va[:, BRANFT[:, 1]]
    vio_branangnum = torch.zeros(Pred_V.shape[0])
    vio_branpfnum = torch.zeros(Pred_V.shape[0])
    vio_branpfidx = torch.zeros(Pred_V.shape[0])
    lsSf = []
    lsSt = []
    lsSf_sampidx = []
    lsSt_sampidx = []
    deltapf = np.array([[0, 0]])
    for i in range(Pred_V.shape[0]):
    #for i in range(1):
        vio_branangnum[i] = np.size(np.where(Pred_branang[i, :] - angminmax[:,0] < -DELTA)) \
        + np.size(np.where(Pred_branang[i, :] - angminmax[:,1] > DELTA))

        # branch power flow
        fV = Pred_V[i, BRANFT[:, 0]]
        tV = Pred_V[i, BRANFT[:, 1]]        
        fI = Yf.dot(Pred_V[i]).conj()
        tI = Yt.dot(Pred_V[i]).conj()
        fS = np.multiply(fV, fI)
        tS = np.multiply(tV, tI)
        deltafS = np.abs(fS) - branlp
        deltatS = np.abs(tS) - branlp
        deltafS = np.array(deltafS).ravel()
        deltatS = np.array(deltatS).ravel()
        idxfs = np.array(np.where(deltafS > DELTA)).reshape(-1, 1)
        idxts = np.array(np.where(deltatS > DELTA)).reshape(-1, 1)
        vio_branpfnum[i] = np.size(idxfs) + np.size(idxfs)
        if np.size(idxfs) >= 1:           
            ii = np.concatenate((idxfs,deltafS[idxfs]),axis=1)  
            deltapf = np.append(deltapf, ii, axis = 0) 
            ii = np.concatenate((ii,np.real(fS[idxfs]),np.imag(fS[idxfs])),axis=1)
            lsSf.append(ii)
            lsSf_sampidx.append(i)           
            
        if np.size(idxts) >= 1:
            ii = np.concatenate((idxts,deltatS[idxts]),axis=1)
            deltapf = np.append(deltapf, ii, axis = 0) 
            # print('ii',ii.shape,'tS',tS.shape,'idxts',idxts.shape)
            ii = np.concatenate((ii,np.real(tS[idxts]),np.imag(tS[idxts])),axis=1)
            lsSt.append(ii)
            lsSt_sampidx.append(i)

        if np.size(idxfs) + np.size(idxts) >= 1:
            vio_branpfidx[i] = i+1
            
    # delete initial
    if deltapf.shape[0] > 1:
        deltapf = np.delete(deltapf, 0, axis = 0)
        deltapfR = deltapf[:, 1]/branlp[0, deltapf[:, 0].astype(int)]*100
        deltapf = np.insert(deltapf, 2, values=deltapfR, axis=1)
 
    vio_branang = (1-vio_branangnum/branch.shape[0])*100 
    vio_branpf = (1-vio_branpfnum/(branch.shape[0]*2))*100
    return vio_branang,vio_branpf,deltapf,vio_branpfidx,lsSf,lsSt,lsSf_sampidx,lsSt_sampidx

def dSlbus_dV(his_V,bus_Va):
    V = his_V.copy()
    # branch power flow for FROM branch
    fV = V[BRANFT[:, 0]]
    fI = Yf.dot(V).conj()
    Nsam = 1
    dfP_dVm = np.zeros((Nsam, branch.shape[0], Nbus))   
    dfQ_dVm = np.zeros((Nsam, branch.shape[0], Nbus))
    dfP_dVa = np.zeros((Nsam, branch.shape[0], Nbus))    
    dfQ_dVa = np.zeros((Nsam, branch.shape[0], Nbus))                
    diagfI = np.diag(fI)
    diagfV = np.diag(fV)
    diagVnorm = np.diag(np.true_divide(V, np.abs(V)))
#     print('diagfV',diagfV.shape,'diagVnorm',diagVnorm.shape,'Yf',Yf.shape,'diagfI',diagfI.shape,'finc',finc.shape)
    dfS_dVm = np.dot(diagfV, Yf.dot(diagVnorm).conj()) + np.dot(diagfI.conj(), np.dot(finc,diagVnorm))
    dfP_dVm = np.real(dfS_dVm)
    dfQ_dVm = np.imag(dfS_dVm)
    diagV = np.diag(V)
    dfS_dVa = -1j*np.dot(diagfV, Yf.dot(diagV).conj()) + 1j*np.dot(diagfI.conj(), np.dot(finc, diagV))
    dfP_dVa = np.real(dfS_dVa)
    dfQ_dVa = np.imag(dfS_dVa)
    dPfbus_dV = np.concatenate((dfP_dVa, dfP_dVm), axis = 1)
    dQfbus_dV = np.concatenate((dfQ_dVa, dfQ_dVm), axis = 1)

    return dPfbus_dV, dQfbus_dV

In [15]:
# revision
# modify V theta
# dPbus_dV dQbus_dV
def dPQbus_dV(his_V,bus_Pg,bus_Qg):
    V = his_V.copy()
#     Ibus = np.dot(Ybus, his_V)
    Ibus = Ybus.dot(his_V).conj()
    diagV = np.diag(V)
    diagIbus = np.diag(Ibus)
    diagVnorm = np.diag(V/np.abs(V))
    dSbus_dVm = np.dot(diagV, Ybus.dot(diagVnorm).conj()) + np.dot(diagIbus.conj(), diagVnorm)
    dSbus_dVa = 1j*np.dot(diagV, (diagIbus - Ybus.dot(diagV)).conj())
    dSbus_dV = np.concatenate((dSbus_dVa, dSbus_dVm), axis=1)
    dPbus_dV = np.real(dSbus_dV)
    dQbus_dV = np.imag(dSbus_dV)
    
    return dPbus_dV,dQbus_dV

# calculate dV
def get_hisdV(lsPg,lsQg,lsidxPg,lsidxQg,num_viotest,k_dV,bus_Pg,bus_Qg,dPbus_dV,dQbus_dV):
    dV = np.zeros((num_viotest,Nbus*2))
    j = 0
    for i in range(Ntest):
        # determin whether there is violation
        if (lsidxPg[i] + lsidxQg[i]) > 0:
            # calculate dS_dV, dPbus_dV,dQbus_dV
            # get BusPg
            if lsidxPg[i] > 0 and lsidxQg[i] > 0:
                idxPg = lsPg[lsidxPg[i]-1][:,0].astype(np.int32) #note: lsidxPg[i]-1
                idxQg = lsQg[lsidxQg[i]-1][:,0].astype(np.int32) #note: lsidxQg[i]-1
                busPg = bus_Pg[idxPg]
                busQg = bus_Qg[idxQg]
                dPGbus_dV = dPbus_dV[busPg, :]
                dQGbus_dV = dQbus_dV[busQg, :]

                if busPg.shape[0] == 1:
                    dPGbus_dV = dPGbus_dV.reshape(1, -1)
                    
                if busQg.shape[0] == 1:
                    dQGbus_dV = dQGbus_dV.reshape(1, -1)               
                
                dPQGbus_dV = np.concatenate((dPGbus_dV, dQGbus_dV), axis=0) #need bus number pf Pg Qg
                dPQg = np.concatenate((lsPg[lsidxPg[i]-1][:,1], lsQg[lsidxQg[i]-1][:,1]), axis=0)
            elif lsidxPg[i] > 0:  
                idxPg = lsPg[lsidxPg[i]-1][:,0].astype(np.int32)
                busPg = bus_Pg[idxPg]
                dPQGbus_dV = dPbus_dV[busPg, :]
                dPQg = lsPg[lsidxPg[i]-1][:,1]
            elif lsidxQg[i] > 0:     
                # get BusQg
                idxQg = lsQg[lsidxQg[i]-1][:,0].astype(np.int32)
                busQg = bus_Qg[idxQg]
                dPQGbus_dV = dQbus_dV[busQg, :]
                dPQg = lsQg[lsidxQg[i]-1][:,1]

            if dPQGbus_dV.ndim > 1:
                dV[j] = np.dot(np.linalg.pinv(dPQGbus_dV), dPQg*k_dV)    
            else:
                idx_nonzero = np.where(dPQGbus_dV > 1e-4)
                dV[j, idx_nonzero] = dPQg*k_dV/dPQGbus_dV[idx_nonzero]
                
            j+=1 
            
    return dV

In [18]:
# paramters load back to CPU
if torch.cuda.is_available():
    MAXMIN_Pg_tensor = MAXMIN_Pg_tensor.cpu()
    MAXMIN_Qg_tensor = MAXMIN_Qg_tensor.cpu()
    
    if flag_test==0:
        bus_Pg,bus_Qg = bus_Pg.cpu(),bus_Qg.cpu()
        bus_Pd,bus_Qd = bus_Pd.cpu(),bus_Qd.cpu()
        bus_nonPg, bus_nonQg = bus_nonPg.cpu(), bus_nonQg.cpu()

    gencost_tensor = gencost_tensor.cpu()
    # rev
    kgenp_max, kgenq_max, kpd_max, kqd_max = kgenp_max.cpu(), kgenq_max.cpu(), kpd_max.cpu(), kqd_max.cpu()
    kcost, kpd, kqd, kgenp, kgenq = kcost.cpu(), kpd.cpu(), kqd.cpu(), kgenp.cpu(), kgenq.cpu()
    kv, kv_max = kv.cpu(),kv_max.cpu()
    VmUb, VmLb = VmUb.cpu(), VmLb.cpu()

In [19]:
# \033[显示方式;前景色;背景色m + 结尾部分：\033[0m#
# print('This is a \033[1;31m test \033[0m!')
import time
start_time = time.process_time()
Pred_Vtest = yvmtest_hat.numpy() * np.exp(1j*yvatest_hat.numpy())
Pred_Pgtest, Pred_Qgtest, Pred_Pdtest, Pred_Qdtest = get_genload(Pred_Vtest, Pdtest, Qdtest, bus_Pg, bus_Qg)
time_cal = (time.process_time() - start_time)
total_time_each = (time_cal + time_NN + time_ZIB)/Ntest
    
# prediction accuracy
# real V, Pd Qg Pd Qd
Real_Vtest = yvmtest.numpy()*np.exp(1j*yvatest.numpy())  
Real_Pgtest, Real_Qgtest, Real_Pdtest, Real_Qdtest = get_genload(Real_Vtest, Pdtest, Qdtest, bus_Pg, bus_Qg) 

# optimality loss
Pred_costtest = get_Pgcost(Pred_Pgtest,idxPg,gencost_Pg)
Real_costtest = get_Pgcost(Real_Pgtest,idxPg,gencost_Pg)
mre_cost = get_rerr2(Real_costtest,Pred_costtest)
# print('Pred_costtest',Pred_costtest.shape, 'Real_costtest', Real_costtest.shape)
# print('mre_cost', mre_cost.shape, 'mean',np.round(np.mean(mre_cost),4),'%  max', np.round(np.max(mre_cost),4), '%  min',np.round(np.min(mre_cost), 4),'%')
print('mre_cost', mre_cost.shape, 'mean\033[1;31m ',str(np.round(np.mean(mre_cost),2)), \
      '\033[0m%  max', np.round(np.max(mre_cost),2), '%  min',np.round(np.min(mre_cost), 2),'%')


# load satisfaction
mre_Pd = 100 - get_rerr(Real_Pdtest.sum(axis=1), Pred_Pdtest.sum(axis=1))
mre_Qd = 100 - get_rerr(Real_Qdtest.sum(axis=1), Pred_Qdtest.sum(axis=1))
# mre_Pd = get_rerr(Real_Pdtest[:, bus_Pd].sum(axis=1), Pred_Pdtest[:, bus_Pd].sum(axis=1))
# mre_Qd = get_rerr(Real_Qdtest[:, bus_Qd].sum(axis=1), Pred_Qdtest[:, bus_Qd].sum(axis=1))
# print('first time: mre_Pd mean', np.round(np.mean(mre_Pd), 4),'%  max', np.round(np.max(mre_Pd),4),'%', \
#      'mre_Qd mean', np.round(np.mean(mre_Qd), 4),'%  max', np.round(np.max(mre_Qd), 4),'%') 
print('first time: mre_Pd mean\033[1;31m ',str(np.round(np.mean(mre_Pd),2)),'\033[0m%  max', np.round(np.max(mre_Pd),2),'%', \
     'mre_Qd mean\033[1;31m ',str(np.round(np.mean(mre_Qd),2)),'\033[0m%  max', np.round(np.max(mre_Qd), 2),'%') 


#calculate violation
# Pg Qg violation
lsPg,lsQg,lsidxPg,lsidxQg,vio_PQgmaxmin,vio_PQg,deltaPgL,deltaPgU,deltaQgL,deltaQgU = get_vioPQg(Pred_Pgtest, bus_Pg, MAXMIN_Pg, Pred_Qgtest, bus_Qg, MAXMIN_Qg)
lsidxPQg = np.squeeze(np.array(np.where((lsidxPg + lsidxQg) > 0)))
num_viotest = np.size(lsidxPQg)
# print('vio_PQg max', torch.max(vio_PQg), 'min', torch.min(vio_PQg), 'mean', torch.mean(vio_PQg), '%',' num_viotest',str(num_viotest),'mean P Q',torch.mean(vio_PQg, 0))
print('vio_PQg mean', torch.mean(vio_PQg), 'min', torch.min(vio_PQg), 'max', torch.max(vio_PQg), '%', \
      ' num_viotest',str(num_viotest),'Pg mean\033[1;31m ',str(np.round(np.mean(vio_PQg[:, 0].numpy()),2)), ' \033[0m%', \
      'Qg mean\033[1;31m ',str(np.round(np.mean(vio_PQg[:, 1].numpy()),2)),'\033[0m%')

# revise power flow (from)
vio_branang,vio_branpf,deltapf,vio_branpfidx,lsSf,_,lsSf_sampidx,_ = get_viobran2(Pred_Vtest,yvatest_hat.numpy(),branch,Yf,Yt)
vio_branpf_num = np.size(np.where(vio_branpfidx>0))
lsSf_sampidx = np.asarray(lsSf_sampidx)
# print('first time: vio_branang: mean', torch.mean(vio_branang), '%  max', torch.max(vio_branang),'% ', \
#       'first time: vio_branpf: mean',torch.mean(vio_branpf), '%  max', torch.max(vio_branpf),'% ')
print('first time: vio_branang: mean\033[1;31m ',str(np.round(np.mean(vio_branang.numpy()),2)), '\033[0m% max', np.max(vio_branang.numpy()),'% ', \
      'first time: vio_branpf: mean\033[1;31m ',str(np.round(np.mean(vio_branpf.numpy()),2)), '\033[0m% max', np.max(vio_branpf.numpy()),'% ')

res_PQUL = np.array((np.mean(deltaPgL[:,1]),np.min(deltaPgL[:,1]),np.mean(deltaPgU[:,1]),np.max(deltaPgU[:,1]), \
                        np.mean(deltaQgL[:,1]),np.min(deltaQgL[:,1]),np.mean(deltaQgU[:,1]),np.max(deltaQgU[:,1])))

if deltapf.shape[1] > 2:
    res_branpf = np.array((np.mean(deltapf[:,1]),np.max(deltapf[:,1]),np.mean(deltapf[:,2]),np.max(deltapf[:,2])))
else:
    res_branpf = np.array((np.mean(deltapf[:,1]),np.max(deltapf[:,1])))
    
print('first time: res_branpf ', res_branpf, 'res_PQUL', res_PQUL)   

# historical V
k_dV = 0.1
print('*'*5,'revision','*'*5)
print('his_V',his_V.shape,type(his_V))
# for justification of Vm Va
dPbus_dV,dQbus_dV = dPQbus_dV(his_V,bus_Pg,bus_Qg)
print('dPbus_dV',dPbus_dV.shape, 'dQbus_dV', dQbus_dV.shape)
dV1 = get_hisdV(lsPg,lsQg,lsidxPg,lsidxQg,num_viotest,k_dV,bus_Pg,bus_Qg,dPbus_dV,dQbus_dV)

# branch
dPfbus_dV, dQfbus_dV = dSlbus_dV(his_V,bus_Va)
print('dPfbus_dV',dPfbus_dV.shape, 'dQfbus_dV', dQfbus_dV.shape)
revision_time_branch = 0
if vio_branpf_num > 0:
    start_time = time.process_time()
    dV_branch = np.zeros((lsSf_sampidx.shape[0], Nbus*2))      
    for i in range(lsSf_sampidx.shape[0]):
        # Pl/deltaSl
        mp = np.array(lsSf[i][:, 2]/lsSf[i][:, 1]).reshape(-1, 1)
        mq = np.array(lsSf[i][:, 3]/lsSf[i][:, 1]).reshape(-1, 1)
        # dPf_dVaVm
        dPdV = dPfbus_dV[np.array(lsSf[i][:, 0].astype(int)).squeeze(), :] 
        # dQf_dVaVm
        dQdV = dQfbus_dV[np.array(lsSf[i][:, 0].astype(int)).squeeze(), :]
        dmp = mp*dPdV 
        dmq = mq*dQdV
        # M: dS_dVaVm
        dmpq_inv = np.linalg.pinv(dmp+dmq)
        dV_branch[i] = np.dot(dmpq_inv, np.array(lsSf[i][:, 1])).squeeze() 
        
    revision_time_branch = (time.process_time() - start_time)/Ntest
    print('revision_time_branch each',revision_time_branch)

start_time = time.process_time()
Pred_Vm1 = yvmtest_hat.clone().numpy()
Pred_Va1 = yvatest_hat.clone().numpy()
# if lsidxPQg.shape[0] > 0: 
if  np.sum(lsidxPQg) > 0: 
    Pred_Va1[lsidxPQg, :] = yvatest_hat[lsidxPQg, :].numpy() - dV1[:, 0:Nbus] # Pg Qg violation
    Pred_Vm1[lsidxPQg, :] = yvmtest_hat[lsidxPQg, :].numpy() - dV1[:, Nbus:2*Nbus] # Pg Qg violation
    
if lsSf_sampidx.shape[0] > 0:    
    Pred_Va1[lsSf_sampidx, :] = Pred_Va1[lsSf_sampidx, :] - dV_branch[:, 0:Nbus] # branch violation     
    Pred_Vm1[lsSf_sampidx, :] = Pred_Vm1[lsSf_sampidx, :] - dV_branch[:, Nbus:2*Nbus] # branch violation
    
Pred_Va1[:,bus_slack] = 0 
Pred_Vm1_clip = get_clamp(torch.from_numpy(Pred_Vm1), VmLb[0].item(), VmUb[0].item())
Pred_Vtest1 = Pred_Vm1_clip.numpy() * np.exp(1j*Pred_Va1)
Pred_Pgtest1, Pred_Qgtest1, Pred_Pdtest1, Pred_Qdtest1 = get_genload(Pred_Vtest1, Pdtest, Qdtest, bus_Pg, bus_Qg)
revision_time_bus = (time.process_time() - start_time)/Ntest
print('revision_time_bus each',revision_time_bus)

###################################################################
# prediction accuracy after revision
mae_Vatest1 = get_mae(yvatest.numpy(), Pred_Va1)
mre_Vatest1 = get_rerr(yvatest[:,bus_Va].numpy(), Pred_Va1[:,bus_Va])
print('mae_Vatest1', mae_Vatest1,'mre_Vatest: min', np.min(mre_Vatest1), \
      'max', np.max(mre_Vatest1), 'mean', np.mean(mre_Vatest1))

mae_Vmtest1 = get_mae(yvmtest.numpy(), Pred_Vm1_clip.numpy())
mre_Vmtest1 = get_rerr(yvmtest.numpy(), Pred_Vm1_clip.numpy())
print('mae_Vmtest1', mae_Vmtest1,'mre_Vmtest1: min', np.min(mre_Vmtest1), \
      'max', np.max(mre_Vmtest1), 'mean', np.mean(mre_Vmtest1))

# optimality loss
Pred_costtest1 = get_Pgcost(Pred_Pgtest1,idxPg,gencost_Pg)
mre_cost1 = get_rerr2(Real_costtest,Pred_costtest1)
# print('mre_cost1', mre_cost1.shape, 'mean',np.round(np.mean(mre_cost1),2),'%  max', np.round(np.max(mre_cost1),4), '%  min',np.round(np.min(mre_cost1), 2),'%')
print('mre_cost1', mre_cost.shape, 'mean\033[1;31m ',str(np.round(np.mean(mre_cost1),2)), \
      '\033[0m%  max', np.round(np.max(mre_cost1),2), '%  min',np.round(np.min(mre_cost1), 2),'%')

mre_Pd1 = 100 - get_rerr(Real_Pdtest.sum(axis=1), Pred_Pdtest1.sum(axis=1))
mre_Qd1 = 100 - get_rerr(Real_Qdtest.sum(axis=1), Pred_Qdtest1.sum(axis=1))
# mre_Pd1 = get_rerr(Real_Pdtest[:, bus_Pd].sum(axis=1), Pred_Pdtest1[:, bus_Pd].sum(axis=1))
# mre_Qd1 = get_rerr(Real_Qdtest[:, bus_Qd].sum(axis=1), Pred_Qdtest1[:, bus_Qd].sum(axis=1))
# print('after revision: mre_Pd1 mean', np.round(np.mean(mre_Pd1), 2),'%  max', np.round(np.max(mre_Pd1),2),'%', \
#      'mre_Qd1 mean', np.round(np.mean(mre_Qd1), 2),'%  max', np.round(np.max(mre_Qd1), 2),'%') 
print('after revision: mre_Pd1 mean\033[1;31m ',str(np.round(np.mean(mre_Pd1),2)),'\033[0m%  max', np.round(np.max(mre_Pd1),2),'%', \
     'mre_Qd1 mean\033[1;31m ',str(np.round(np.mean(mre_Qd1),2)),' \033[0m%  max', np.round(np.max(mre_Qd1), 2),'%') 


#calculate violation
# Pg Qg violation
_,_,lsidxPg1,lsidxQg1,vio_PQgmaxmin1,vio_PQg1,deltaPgL1,deltaPgU1,deltaQgL1,deltaQgU1 = get_vioPQg(Pred_Pgtest1, bus_Pg, MAXMIN_Pg, Pred_Qgtest1, bus_Qg, MAXMIN_Qg)
lsidxPQg1 = np.squeeze(np.array(np.where((lsidxPg1 + lsidxQg1) > 0)))
num_viotest1 = np.size(lsidxPQg1)
# print('vio_PQg1 max', torch.max(vio_PQg1), 'min', torch.min(vio_PQg1), 'mean', torch.mean(vio_PQg1), '%',' num_viotest',str(num_viotest1),'mean P Q',torch.mean(vio_PQg1, 0))
print('vio_PQg1 mean', torch.mean(vio_PQg1), 'min', torch.min(vio_PQg), 'max', torch.max(vio_PQg1), '%', \
      ' num_viotest',str(num_viotest1),'Pg mean\033[1;31m ',str(np.round(np.mean(vio_PQg1[:, 0].numpy()),2)), \
      ' \033[0m%','Qg mean\033[1;31m ',str(np.round(np.mean(vio_PQg1[:, 1].numpy()),4)),'\033[0m%')


# branch
vio_branang1,vio_branpf1,deltapf1,vio_branpfidx1,lsSf1,_,lsSf_sampidx1,_ = get_viobran2(Pred_Vtest1,yvatest_hat.numpy(),branch,Yf,Yt)
# print('after revision: vio_branang1: mean', torch.mean(vio_branang1), '%  max', torch.max(vio_branang1),'% ', \
#       'vio_branpf1: mean',torch.mean(vio_branpf1), '%  max', torch.max(vio_branpf1),'% ')
print('after revision: vio_branang1: mean\033[1;31m ',str(np.round(np.mean(vio_branang1.numpy()),2)), '\033[0m% max', np.max(vio_branang1.numpy()),'% ', \
      'after revision: vio_branpf1: mean\033[1;31m ',str(np.round(np.mean(vio_branpf1.numpy()),2)), '\033[0m% max', np.max(vio_branpf1.numpy()),'% ')


res_PQUL1 = np.array((np.mean(deltaPgL1[:,1]),np.min(deltaPgL1[:,1]),np.mean(deltaPgU1[:,1]),np.max(deltaPgU1[:,1]), \
                        np.mean(deltaQgL1[:,1]),np.min(deltaQgL1[:,1]),np.mean(deltaQgU1[:,1]),np.max(deltaQgU1[:,1])))

if deltapf1.shape[1] > 2:
    res_branpf1 = np.array((np.mean(deltapf1[:,1]),np.max(deltapf1[:,1]),np.mean(deltapf1[:,2]),np.max(deltapf1[:,2])))
else:
    res_branpf1 = np.array((np.mean(deltapf1[:,1]),np.max(deltapf1[:,1])))
    
print('after revision: res_branpf1 ', res_branpf1, 'res_PQUL1', res_PQUL1)   

# time
total_time_each_rev = total_time_each + revision_time_bus + revision_time_branch
print('total_time_each', np.round(total_time_each,6))
print('total_time_each_rev', np.round(total_time_each_rev,6))
    
optimality_loss = np.mean(np.abs((Pred_costtest-Real_costtest)/Real_costtest))*100
print('optimality_loss',optimality_loss,'%')

mre_cost (10000,) mean  -31.53 %  max -22.32 %  min -40.36 %
first time: mre_Pd mean  60.71 %  max 64.81 % mre_Qd mean  76.23 %  max 95.31 %
vio_PQg mean tensor(67.3267) min tensor(66.6667) max tensor(83.3333) %  num_viotest 10000 Pg mean  66.67  % Qg mean  67.99 %
first time: vio_branang: mean  100.0 % max 100.0 %  first time: vio_branpf: mean  100.0 % max 100.0 % 
first time: res_branpf  [0.001  0.002  0.3136 0.6381] res_PQUL [-0.8738 -0.922   0.7598  0.7844 -0.1208 -0.199   0.0957  0.1551]
***** revision *****
his_V (30,) <class 'numpy.ndarray'>
dPbus_dV (30, 60) dQbus_dV (30, 60)
dPfbus_dV (41, 60) dQfbus_dV (41, 60)
revision_time_branch each 0.0
revision_time_bus each 0.0003125
mae_Vatest1 0.015111748 mre_Vatest: min 0.0 max 1497.2327 mean 51.776474
mae_Vmtest1 0.000491954 mre_Vmtest1: min 0.0 max 3.157898 mean 0.05004616
mre_cost1 (10000,) mean  -30.72 %  max -21.1 %  min -39.56 %
after revision: mre_Pd1 mean  62.17 %  max 66.82 % mre_Qd1 mean  76.18  %  max 95.8 %
vio_PQg1 mean 

In [20]:
if flag_test:
    # save results
    res_tensor = torch.as_tensor([mae_Vmtest, mae_Vatest, np.mean(mre_cost), np.mean(mre_Pd), np.mean(mre_Qd), \
                    torch.mean(vio_PQg[:, 0]), torch.mean(vio_PQg[:, 1]), torch.mean(vio_branang), torch.mean(vio_branpf)]).reshape(1,-1)
    res_tensor1 = torch.as_tensor([mae_Vmtest1, mae_Vatest1, np.mean(mre_cost1), np.mean(mre_Pd1), np.mean(mre_Qd1), \
                    torch.mean(vio_PQg1[:, 0]), torch.mean(vio_PQg1[:, 1]), torch.mean(vio_branang1), torch.mean(vio_branpf1)]).reshape(1,-1)
    res = np.around(torch.cat((res_tensor, res_tensor1), axis = 1).numpy(), decimals=2)
    mre_cost = np.array(mre_cost)
    time = np.array([total_time_each, total_time_each_rev])
    scipy.io.savemat(resultnm, 
                     mdict={'res': res,'res_PQUL': res_PQUL,'deltaPgL': deltaPgL,'deltaPgU': deltaPgU,'deltaQgL': deltaQgL, \
                            'deltaQgU': deltaQgU,'time_ZIB': time_ZIB,'time_NN': time_NN,'time_cal': time_cal,'total_time_each':total_time_each, \
                            'total_time_each_rev': total_time_each_rev,'revision_time_bus': revision_time_bus, \
                            'res_branpf':res_branpf,'mre_cost':mre_cost,'deltapf':deltapf,'deltapf1':deltapf1,\
                            'idx_train':idx_train,'idx_test':idx_test, \
                            'idx_his': idx_his, 'res_PQUL1': res_PQUL1,'deltaPgL1': deltaPgL1,'deltaPgU1': deltaPgU1,'deltaQgL1': deltaQgL1, \
                            'deltaQgU1': deltaQgU1, 'time': time,'optimality_loss':optimality_loss
    })
    
else:
        # save results
    res_tensor = torch.as_tensor([mae_Vmtest, mae_Vatest, np.mean(mre_cost), np.mean(mre_Pd), np.mean(mre_Qd), \
                    torch.mean(vio_PQg[:, 0]), torch.mean(vio_PQg[:, 1]), torch.mean(vio_branang), torch.mean(vio_branpf)]).reshape(1,-1)
    res_tensor1 = torch.as_tensor([mae_Vmtest1, mae_Vatest1, np.mean(mre_cost1), np.mean(mre_Pd1), np.mean(mre_Qd1), \
                    torch.mean(vio_PQg1[:, 0]), torch.mean(vio_PQg1[:, 1]), torch.mean(vio_branang1), torch.mean(vio_branpf1)]).reshape(1,-1)
    res = np.around(torch.cat((res_tensor, res_tensor1), axis = 1).numpy(), decimals=2)
    mre_cost = np.array(mre_cost)
    time = np.array([total_time_each, total_time_each_rev, time_trian])
    scipy.io.savemat(resultnm, 
                     mdict={'res': res,'res_PQUL': res_PQUL,'deltaPgL': deltaPgL,'deltaPgU': deltaPgU,'deltaQgL': deltaQgL, \
                            'deltaQgU': deltaQgU,'time_ZIB': time_ZIB,'time_NN': time_NN,'total_time_each':total_time_each, \
                            'total_time_each_rev': total_time_each_rev,'revision_time_bus': revision_time_bus, \
                            'res_branpf':res_branpf,'mre_cost':mre_cost,'deltapf':deltapf,'deltapf1':deltapf1,\
                            'idx_train':idx_train,'idx_test':idx_test, \
                            'idx_his': idx_his, 'res_PQUL1': res_PQUL1,'deltaPgL1': deltaPgL1,'deltaPgU1': deltaPgU1,'deltaQgL1': deltaQgL1, \
                            'deltaQgU1': deltaQgU1, 'time': time, 'time_trian': time_trian,'optimality_loss':optimality_loss
    })

In [21]:
print('resultnm',resultnm)

resultnm ./result/Res_DeepOPFSB_krp_B30r0V1S32000Nhis3lr0.0001kc0.0kd100kg2000kv500ky1L32224bs50E45001Nt10000.mat


In [22]:
model

NetV(
  (fc1): Linear(in_features=40, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=224, bias=True)
  (fcbfend): Linear(in_features=224, out_features=47, bias=True)
  (fcend): Linear(in_features=47, out_features=47, bias=True)
)